# ETL

# Performing ETL on All Datasets Created From "/data_analysis/data_analysis_cleaned.ipynb"

In [1]:
# Install the needed dependencies
import sys
!{sys.executable} -m pip install python-dotenv
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install geopandas

In [2]:
# Importing the Needed Dependencies

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 400)
import os
import requests
import json
import csv
import geopandas as gpd
from pprint import pprint
from pathlib import Path
import ast

# Performing ETL on "top3_alt_fuels.csv" Dataset

## Top 3 Alternative Fuel Stations Data

In [3]:
# Create reference to CSV file
csv_path = Path("../output_data/data_analysis/csv/top3_alt_fuels.csv")

# Import the CSV into a pandas DataFrame
top3_alt_fuels_df = pd.read_csv(csv_path)
top3_alt_fuels_df

,id,state,coordinates,fuel type
0,1517,CA,"[-118.3879713743439, 34.2483191527193]",ELEC
1,1519,CA,"[-118.448504, 34.052542]",ELEC
2,1523,CA,"[-118.271387, 34.040539]",ELEC
3,1525,CA,"[-118.248589, 34.059133]",ELEC
4,1531,CA,"[-118.096665, 33.759802]",ELEC
...,...,...,...,...
25121,306460,TX,"[-95.98519, 31.25954]",ELEC
25122,306461,TX,"[-95.98519, 31.25951]",ELEC
25123,306462,TX,"[-95.98519, 31.25942]",ELEC
25124,306463,TX,"[-95.985225, 31.25934]",ELEC


In [4]:
# Get a brief summary of the top3_alt_fuels_df DataFrame.
top3_alt_fuels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25126 entries, 0 to 25125
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           25126 non-null  int64 
 1   state        25126 non-null  object
 2   coordinates  25126 non-null  object
 3   fuel type    25126 non-null  object
dtypes: int64(1), object(3)
memory usage: 785.3+ KB


In [5]:
# converting column values to string
# source: https://docs.python.org/3/library/ast.html#ast.literal_eval
top3_alt_fuels_df['coordinates'] = top3_alt_fuels_df['coordinates'].map(lambda x: ast.literal_eval(x))

In [6]:
# Rename the id, state, coordinates and fuel type columns.
top3_alt_fuels_df = top3_alt_fuels_df.rename(columns={"id":"ID", "state":"State", 
                                                      "coordinates":"Lon & Lat", "fuel type":"Fuel Type"})
top3_alt_fuels_df.head()

,ID,State,Lon & Lat,Fuel Type
0,1517,CA,"[-118.3879713743439, 34.2483191527193]",ELEC
1,1519,CA,"[-118.448504, 34.052542]",ELEC
2,1523,CA,"[-118.271387, 34.040539]",ELEC
3,1525,CA,"[-118.248589, 34.059133]",ELEC
4,1531,CA,"[-118.096665, 33.759802]",ELEC


In [7]:
# Removing '[' from the beginning of 'Lat' values and ']' from the end of 'lon' values
for i, v in top3_alt_fuels_df.iterrows():
    print(v['Lon & Lat'][0], v['Lon & Lat'][1])
    if i == 5:
        break;

-118.3879713743439 34.2483191527193
-118.448504 34.052542
-118.271387 34.040539
-118.248589 34.059133
-118.096665 33.759802
-118.265628 33.770508


In [8]:
# Separating the "Lat & Lon" column into the two columns of "latitude" and "longitude"
top3_alt_fuels_df['Longitude'] = top3_alt_fuels_df['Lon & Lat'].map(lambda x: x[0])
top3_alt_fuels_df['Latitude'] = top3_alt_fuels_df['Lon & Lat'].map(lambda x: x[1])
top3_alt_fuels_df

,ID,State,Lon & Lat,Fuel Type,Longitude,Latitude
0,1517,CA,"[-118.3879713743439, 34.2483191527193]",ELEC,-118.387971,34.248319
1,1519,CA,"[-118.448504, 34.052542]",ELEC,-118.448504,34.052542
2,1523,CA,"[-118.271387, 34.040539]",ELEC,-118.271387,34.040539
3,1525,CA,"[-118.248589, 34.059133]",ELEC,-118.248589,34.059133
4,1531,CA,"[-118.096665, 33.759802]",ELEC,-118.096665,33.759802
...,...,...,...,...,...,...
25121,306460,TX,"[-95.98519, 31.25954]",ELEC,-95.985190,31.259540
25122,306461,TX,"[-95.98519, 31.25951]",ELEC,-95.985190,31.259510
25123,306462,TX,"[-95.98519, 31.25942]",ELEC,-95.985190,31.259420
25124,306463,TX,"[-95.985225, 31.25934]",ELEC,-95.985225,31.259340


In [9]:
# Deleting the "Lon & Lat" column
top3_alt_fuels_df = top3_alt_fuels_df.drop('Lon & Lat', axis=1)
top3_alt_fuels_df

,ID,State,Fuel Type,Longitude,Latitude
0,1517,CA,ELEC,-118.387971,34.248319
1,1519,CA,ELEC,-118.448504,34.052542
2,1523,CA,ELEC,-118.271387,34.040539
3,1525,CA,ELEC,-118.248589,34.059133
4,1531,CA,ELEC,-118.096665,33.759802
...,...,...,...,...,...
25121,306460,TX,ELEC,-95.985190,31.259540
25122,306461,TX,ELEC,-95.985190,31.259510
25123,306462,TX,ELEC,-95.985190,31.259420
25124,306463,TX,ELEC,-95.985225,31.259340


In [10]:
# Changing order of "Longitude" and "Latitude" columns to "Latitude" and "Longitude"
top3_alt_fuels_df = top3_alt_fuels_df.reindex(['ID','State','Fuel Type','Latitude', 'Longitude'], axis=1)
top3_alt_fuels_df

,ID,State,Fuel Type,Latitude,Longitude
0,1517,CA,ELEC,34.248319,-118.387971
1,1519,CA,ELEC,34.052542,-118.448504
2,1523,CA,ELEC,34.040539,-118.271387
3,1525,CA,ELEC,34.059133,-118.248589
4,1531,CA,ELEC,33.759802,-118.096665
...,...,...,...,...,...
25121,306460,TX,ELEC,31.259540,-95.985190
25122,306461,TX,ELEC,31.259510,-95.985190
25123,306462,TX,ELEC,31.259420,-95.985190
25124,306463,TX,ELEC,31.259340,-95.985225


In [11]:
# renaming "ELEC" to "Electric", "E85" to "Ethanol" and "LPG" to "Propane" in the "Fuel Type" column
top3_alt_fuels_df['Fuel Type'] = top3_alt_fuels_df['Fuel Type'].replace({'ELEC': 'Electric'})
top3_alt_fuels_df['Fuel Type'] = top3_alt_fuels_df['Fuel Type'].replace({'E85': 'Ethanol'})
top3_alt_fuels_df['Fuel Type'] = top3_alt_fuels_df['Fuel Type'].replace({'LPG': 'Propane'})
top3_alt_fuels_df

,ID,State,Fuel Type,Latitude,Longitude
0,1517,CA,Electric,34.248319,-118.387971
1,1519,CA,Electric,34.052542,-118.448504
2,1523,CA,Electric,34.040539,-118.271387
3,1525,CA,Electric,34.059133,-118.248589
4,1531,CA,Electric,33.759802,-118.096665
...,...,...,...,...,...
25121,306460,TX,Electric,31.259540,-95.985190
25122,306461,TX,Electric,31.259510,-95.985190
25123,306462,TX,Electric,31.259420,-95.985190
25124,306463,TX,Electric,31.259340,-95.985225


In [12]:
# renaming "CA" to "California", "TX" to "Texas" and "NY" to "New York" in the "State" column
top3_alt_fuels_df['State'] = top3_alt_fuels_df['State'].replace({'CA': 'California'})
top3_alt_fuels_df['State'] = top3_alt_fuels_df['State'].replace({'TX': 'Texas'})
top3_alt_fuels_df['State'] = top3_alt_fuels_df['State'].replace({'NY': 'New York'})
top3_alt_fuels_df

,ID,State,Fuel Type,Latitude,Longitude
0,1517,California,Electric,34.248319,-118.387971
1,1519,California,Electric,34.052542,-118.448504
2,1523,California,Electric,34.040539,-118.271387
3,1525,California,Electric,34.059133,-118.248589
4,1531,California,Electric,33.759802,-118.096665
...,...,...,...,...,...
25121,306460,Texas,Electric,31.259540,-95.985190
25122,306461,Texas,Electric,31.259510,-95.985190
25123,306462,Texas,Electric,31.259420,-95.985190
25124,306463,Texas,Electric,31.259340,-95.985225


### Exporting Transformed Dataframe to CSV

In [13]:
# Export the newly transformed dataframe into a csv
top3_alt_fuels_df.to_csv("../output_data/ETL/csv/ETL_top3_alt_fuels.csv", index_label="#")

# Performing ETL on "top3_alt_fuels_CA.csv" Dataset

## Top 3 Alternative Fuel Stations Data for California

In [14]:
# Create reference to CSV file
csv_path = Path("../output_data/data_analysis/csv/top3_alt_fuels_CA.csv")

# Import the CSV into a pandas DataFrame
top3_alt_fuels_CA_df = pd.read_csv(csv_path)
top3_alt_fuels_CA_df

,id,state,coordinates,fuel type
0,1517,CA,"[-118.3879713743439, 34.2483191527193]",ELEC
1,1519,CA,"[-118.448504, 34.052542]",ELEC
2,1523,CA,"[-118.271387, 34.040539]",ELEC
3,1525,CA,"[-118.248589, 34.059133]",ELEC
4,1531,CA,"[-118.096665, 33.759802]",ELEC
...,...,...,...,...
17322,306367,CA,"[-116.969808, 33.209603]",ELEC
17323,306368,CA,"[-116.969739, 33.209602]",ELEC
17324,306408,CA,"[-119.28747, 36.32828]",ELEC
17325,306414,CA,"[-116.969933, 33.20959]",ELEC


In [15]:
# Get a brief summary of the top3_alt_fuels_CA_df DataFrame.
top3_alt_fuels_CA_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17327 entries, 0 to 17326
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           17327 non-null  int64 
 1   state        17327 non-null  object
 2   coordinates  17327 non-null  object
 3   fuel type    17327 non-null  object
dtypes: int64(1), object(3)
memory usage: 541.6+ KB


In [16]:
# converting column values to string
# source: https://docs.python.org/3/library/ast.html#ast.literal_eval
top3_alt_fuels_CA_df['coordinates'] = top3_alt_fuels_CA_df['coordinates'].map(lambda x: ast.literal_eval(x))

In [17]:
# Rename the id, state, coordinates and fuel type columns.
top3_alt_fuels_CA_df = top3_alt_fuels_CA_df.rename(columns={"id":"ID", "state":"State", 
                                                            "coordinates":"Lon & Lat", "fuel type":"Fuel Type"})
top3_alt_fuels_CA_df.head()

,ID,State,Lon & Lat,Fuel Type
0,1517,CA,"[-118.3879713743439, 34.2483191527193]",ELEC
1,1519,CA,"[-118.448504, 34.052542]",ELEC
2,1523,CA,"[-118.271387, 34.040539]",ELEC
3,1525,CA,"[-118.248589, 34.059133]",ELEC
4,1531,CA,"[-118.096665, 33.759802]",ELEC


In [18]:
# Removing '[' from the beginning of 'Lat' values and ']' from the end of 'lon' values
for i, v in top3_alt_fuels_CA_df.iterrows():
    print(v['Lon & Lat'][0], v['Lon & Lat'][1])
    if i == 5:
        break;

-118.3879713743439 34.2483191527193
-118.448504 34.052542
-118.271387 34.040539
-118.248589 34.059133
-118.096665 33.759802
-118.265628 33.770508


In [19]:
# Separating the "Lat & Lon" column into the two columns of "latitude" and "longitude"
top3_alt_fuels_CA_df['Longitude'] = top3_alt_fuels_CA_df['Lon & Lat'].map(lambda x: x[0])
top3_alt_fuels_CA_df['Latitude'] = top3_alt_fuels_CA_df['Lon & Lat'].map(lambda x: x[1])
top3_alt_fuels_CA_df

,ID,State,Lon & Lat,Fuel Type,Longitude,Latitude
0,1517,CA,"[-118.3879713743439, 34.2483191527193]",ELEC,-118.387971,34.248319
1,1519,CA,"[-118.448504, 34.052542]",ELEC,-118.448504,34.052542
2,1523,CA,"[-118.271387, 34.040539]",ELEC,-118.271387,34.040539
3,1525,CA,"[-118.248589, 34.059133]",ELEC,-118.248589,34.059133
4,1531,CA,"[-118.096665, 33.759802]",ELEC,-118.096665,33.759802
...,...,...,...,...,...,...
17322,306367,CA,"[-116.969808, 33.209603]",ELEC,-116.969808,33.209603
17323,306368,CA,"[-116.969739, 33.209602]",ELEC,-116.969739,33.209602
17324,306408,CA,"[-119.28747, 36.32828]",ELEC,-119.287470,36.328280
17325,306414,CA,"[-116.969933, 33.20959]",ELEC,-116.969933,33.209590


In [20]:
# Deleting the "Lat & Lon" column
top3_alt_fuels_CA_df = top3_alt_fuels_CA_df.drop('Lon & Lat', axis=1)
top3_alt_fuels_CA_df

,ID,State,Fuel Type,Longitude,Latitude
0,1517,CA,ELEC,-118.387971,34.248319
1,1519,CA,ELEC,-118.448504,34.052542
2,1523,CA,ELEC,-118.271387,34.040539
3,1525,CA,ELEC,-118.248589,34.059133
4,1531,CA,ELEC,-118.096665,33.759802
...,...,...,...,...,...
17322,306367,CA,ELEC,-116.969808,33.209603
17323,306368,CA,ELEC,-116.969739,33.209602
17324,306408,CA,ELEC,-119.287470,36.328280
17325,306414,CA,ELEC,-116.969933,33.209590


In [21]:
# Changing order of "Longitude" and "Latitude" columns to "Latitude" and "Longitude"
top3_alt_fuels_CA_df = top3_alt_fuels_CA_df.reindex(['ID','State','Fuel Type','Latitude', 'Longitude'], axis=1)
top3_alt_fuels_CA_df

,ID,State,Fuel Type,Latitude,Longitude
0,1517,CA,ELEC,34.248319,-118.387971
1,1519,CA,ELEC,34.052542,-118.448504
2,1523,CA,ELEC,34.040539,-118.271387
3,1525,CA,ELEC,34.059133,-118.248589
4,1531,CA,ELEC,33.759802,-118.096665
...,...,...,...,...,...
17322,306367,CA,ELEC,33.209603,-116.969808
17323,306368,CA,ELEC,33.209602,-116.969739
17324,306408,CA,ELEC,36.328280,-119.287470
17325,306414,CA,ELEC,33.209590,-116.969933


In [22]:
# renaming "ELEC" to "Electric", "E85" to "Ethanol" and "LPG" to "Propane" in the "Fuel Type" column
top3_alt_fuels_CA_df['Fuel Type'] = top3_alt_fuels_CA_df['Fuel Type'].replace({'ELEC': 'Electric'})
top3_alt_fuels_CA_df['Fuel Type'] = top3_alt_fuels_CA_df['Fuel Type'].replace({'E85': 'Ethanol'})
top3_alt_fuels_CA_df['Fuel Type'] = top3_alt_fuels_CA_df['Fuel Type'].replace({'LPG': 'Propane'})
top3_alt_fuels_CA_df

,ID,State,Fuel Type,Latitude,Longitude
0,1517,CA,Electric,34.248319,-118.387971
1,1519,CA,Electric,34.052542,-118.448504
2,1523,CA,Electric,34.040539,-118.271387
3,1525,CA,Electric,34.059133,-118.248589
4,1531,CA,Electric,33.759802,-118.096665
...,...,...,...,...,...
17322,306367,CA,Electric,33.209603,-116.969808
17323,306368,CA,Electric,33.209602,-116.969739
17324,306408,CA,Electric,36.328280,-119.287470
17325,306414,CA,Electric,33.209590,-116.969933


In [23]:
# renaming "CA" to "California", "TX" to "Texas" and "NY" to "New York" in the "State" column
top3_alt_fuels_CA_df['State'] = top3_alt_fuels_CA_df['State'].replace({'CA': 'California'})
top3_alt_fuels_CA_df['State'] = top3_alt_fuels_CA_df['State'].replace({'TX': 'Texas'})
top3_alt_fuels_CA_df['State'] = top3_alt_fuels_CA_df['State'].replace({'NY': 'New York'})
top3_alt_fuels_CA_df

,ID,State,Fuel Type,Latitude,Longitude
0,1517,California,Electric,34.248319,-118.387971
1,1519,California,Electric,34.052542,-118.448504
2,1523,California,Electric,34.040539,-118.271387
3,1525,California,Electric,34.059133,-118.248589
4,1531,California,Electric,33.759802,-118.096665
...,...,...,...,...,...
17322,306367,California,Electric,33.209603,-116.969808
17323,306368,California,Electric,33.209602,-116.969739
17324,306408,California,Electric,36.328280,-119.287470
17325,306414,California,Electric,33.209590,-116.969933


### Exporting Transformed Dataframe to CSV

In [24]:
# Export the newly transformed dataframe into a csv
top3_alt_fuels_CA_df.to_csv("../output_data/ETL/csv/ETL_top3_alt_fuels_CA.csv", index_label="#")

# Performing ETL on "E85_CA.csv" Dataset

## California Ethanol Fuel Stations Data

In [25]:
# Create reference to CSV file
csv_path = Path("../output_data/data_analysis/csv/E85_CA.csv")

# Import the CSV into a pandas DataFrame
E85_CA_df = pd.read_csv(csv_path)
E85_CA_df

,id,state,coordinates,fuel type
0,22617,CA,"[-117.107501, 32.754335]",E85
1,32583,CA,"[-117.251427, 33.127936]",E85
2,32584,CA,"[-118.465643, 34.053925]",E85
3,32585,CA,"[-117.364261, 33.189298]",E85
4,32954,CA,"[-121.999551, 38.01483]",E85
...,...,...,...,...
349,302319,CA,"[-118.06816, 34.14753]",E85
350,302674,CA,"[-117.7356403, 34.0114241]",E85
351,302675,CA,"[-117.4038544, 33.93509863]",E85
352,302676,CA,"[-118.2534124, 34.23223656]",E85


In [26]:
# Get a brief summary of the E85_CA_df DataFrame.
E85_CA_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           354 non-null    int64 
 1   state        354 non-null    object
 2   coordinates  354 non-null    object
 3   fuel type    354 non-null    object
dtypes: int64(1), object(3)
memory usage: 11.2+ KB


In [27]:
# converting column values to string
# source: https://docs.python.org/3/library/ast.html#ast.literal_eval
E85_CA_df['coordinates'] = E85_CA_df['coordinates'].map(lambda x: ast.literal_eval(x))

In [28]:
# Rename the id, state, coordinates and fuel type columns.
E85_CA_df = E85_CA_df.rename(columns={"id":"ID", "state":"State", 
                                      "coordinates":"Lon & Lat", "fuel type":"Fuel Type"})
E85_CA_df.head()

,ID,State,Lon & Lat,Fuel Type
0,22617,CA,"[-117.107501, 32.754335]",E85
1,32583,CA,"[-117.251427, 33.127936]",E85
2,32584,CA,"[-118.465643, 34.053925]",E85
3,32585,CA,"[-117.364261, 33.189298]",E85
4,32954,CA,"[-121.999551, 38.01483]",E85


In [29]:
# Removing '[' from the beginning of 'Lat' values and ']' from the end of 'lon' values
for i, v in E85_CA_df.iterrows():
    print(v['Lon & Lat'][0], v['Lon & Lat'][1])
    if i == 5:
        break;

-117.107501 32.754335
-117.251427 33.127936
-118.465643 34.053925
-117.364261 33.189298
-121.999551 38.01483
-122.110511 37.666392


In [30]:
# Separating the "Lat & Lon" column into the two columns of "latitude" and "longitude"
E85_CA_df['Longitude'] = E85_CA_df['Lon & Lat'].map(lambda x: x[0])
E85_CA_df['Latitude'] = E85_CA_df['Lon & Lat'].map(lambda x: x[1])
E85_CA_df

,ID,State,Lon & Lat,Fuel Type,Longitude,Latitude
0,22617,CA,"[-117.107501, 32.754335]",E85,-117.107501,32.754335
1,32583,CA,"[-117.251427, 33.127936]",E85,-117.251427,33.127936
2,32584,CA,"[-118.465643, 34.053925]",E85,-118.465643,34.053925
3,32585,CA,"[-117.364261, 33.189298]",E85,-117.364261,33.189298
4,32954,CA,"[-121.999551, 38.01483]",E85,-121.999551,38.014830
...,...,...,...,...,...,...
349,302319,CA,"[-118.06816, 34.14753]",E85,-118.068160,34.147530
350,302674,CA,"[-117.7356403, 34.0114241]",E85,-117.735640,34.011424
351,302675,CA,"[-117.4038544, 33.93509863]",E85,-117.403854,33.935099
352,302676,CA,"[-118.2534124, 34.23223656]",E85,-118.253412,34.232237


In [31]:
# Deleting the "Lon & Lat" column
E85_CA_df = E85_CA_df.drop('Lon & Lat', axis=1)
E85_CA_df

,ID,State,Fuel Type,Longitude,Latitude
0,22617,CA,E85,-117.107501,32.754335
1,32583,CA,E85,-117.251427,33.127936
2,32584,CA,E85,-118.465643,34.053925
3,32585,CA,E85,-117.364261,33.189298
4,32954,CA,E85,-121.999551,38.014830
...,...,...,...,...,...
349,302319,CA,E85,-118.068160,34.147530
350,302674,CA,E85,-117.735640,34.011424
351,302675,CA,E85,-117.403854,33.935099
352,302676,CA,E85,-118.253412,34.232237


In [32]:
# Changing order of "Longitude" and "Latitude" columns to "Latitude" and "Longitude"
E85_CA_df = E85_CA_df.reindex(['ID','State','Fuel Type','Latitude', 'Longitude'], axis=1)
E85_CA_df

,ID,State,Fuel Type,Latitude,Longitude
0,22617,CA,E85,32.754335,-117.107501
1,32583,CA,E85,33.127936,-117.251427
2,32584,CA,E85,34.053925,-118.465643
3,32585,CA,E85,33.189298,-117.364261
4,32954,CA,E85,38.014830,-121.999551
...,...,...,...,...,...
349,302319,CA,E85,34.147530,-118.068160
350,302674,CA,E85,34.011424,-117.735640
351,302675,CA,E85,33.935099,-117.403854
352,302676,CA,E85,34.232237,-118.253412


In [33]:
# renaming "ELEC" to "Electric", "E85" to "Ethanol" and "LPG" to "Propane" in the "Fuel Type" column
E85_CA_df['Fuel Type'] = E85_CA_df['Fuel Type'].replace({'ELEC': 'Electric'})
E85_CA_df['Fuel Type'] = E85_CA_df['Fuel Type'].replace({'E85': 'Ethanol'})
E85_CA_df['Fuel Type'] = E85_CA_df['Fuel Type'].replace({'LPG': 'Propane'})
E85_CA_df

,ID,State,Fuel Type,Latitude,Longitude
0,22617,CA,Ethanol,32.754335,-117.107501
1,32583,CA,Ethanol,33.127936,-117.251427
2,32584,CA,Ethanol,34.053925,-118.465643
3,32585,CA,Ethanol,33.189298,-117.364261
4,32954,CA,Ethanol,38.014830,-121.999551
...,...,...,...,...,...
349,302319,CA,Ethanol,34.147530,-118.068160
350,302674,CA,Ethanol,34.011424,-117.735640
351,302675,CA,Ethanol,33.935099,-117.403854
352,302676,CA,Ethanol,34.232237,-118.253412


In [34]:
# renaming "CA" to "California", "TX" to "Texas" and "NY" to "New York" in the "State" column
E85_CA_df['State'] = E85_CA_df['State'].replace({'CA': 'California'})
E85_CA_df['State'] = E85_CA_df['State'].replace({'TX': 'Texas'})
E85_CA_df['State'] = E85_CA_df['State'].replace({'NY': 'New York'})
E85_CA_df

,ID,State,Fuel Type,Latitude,Longitude
0,22617,California,Ethanol,32.754335,-117.107501
1,32583,California,Ethanol,33.127936,-117.251427
2,32584,California,Ethanol,34.053925,-118.465643
3,32585,California,Ethanol,33.189298,-117.364261
4,32954,California,Ethanol,38.014830,-121.999551
...,...,...,...,...,...
349,302319,California,Ethanol,34.147530,-118.068160
350,302674,California,Ethanol,34.011424,-117.735640
351,302675,California,Ethanol,33.935099,-117.403854
352,302676,California,Ethanol,34.232237,-118.253412


### Exporting Transformed Dataframe to CSV

In [35]:
# Export the newly transformed dataframe into a csv
E85_CA_df.to_csv("../output_data/ETL/csv/ETL_E85_CA.csv", index_label="#")

# Performing ETL on "ELEC_CA.csv" Dataset

## California Electric Car Charging Stations Data

In [36]:
# Create reference to CSV file
csv_path = Path("../output_data/data_analysis/csv/ELEC_CA.csv")

# Import the CSV into a pandas DataFrame
ELEC_CA_df = pd.read_csv(csv_path)
ELEC_CA_df

,id,state,coordinates,fuel type
0,1517,CA,"[-118.3879713743439, 34.2483191527193]",ELEC
1,1519,CA,"[-118.448504, 34.052542]",ELEC
2,1523,CA,"[-118.271387, 34.040539]",ELEC
3,1525,CA,"[-118.248589, 34.059133]",ELEC
4,1531,CA,"[-118.096665, 33.759802]",ELEC
...,...,...,...,...
16711,306367,CA,"[-116.969808, 33.209603]",ELEC
16712,306368,CA,"[-116.969739, 33.209602]",ELEC
16713,306408,CA,"[-119.28747, 36.32828]",ELEC
16714,306414,CA,"[-116.969933, 33.20959]",ELEC


In [37]:
# Get a brief summary of the ELEC_CA_df DataFrame.
ELEC_CA_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16716 entries, 0 to 16715
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           16716 non-null  int64 
 1   state        16716 non-null  object
 2   coordinates  16716 non-null  object
 3   fuel type    16716 non-null  object
dtypes: int64(1), object(3)
memory usage: 522.5+ KB


In [38]:
# converting column values to string
# source: https://docs.python.org/3/library/ast.html#ast.literal_eval
ELEC_CA_df['coordinates'] = ELEC_CA_df['coordinates'].map(lambda x: ast.literal_eval(x))

In [39]:
# Rename the id, state, coordinates and fuel type columns.
ELEC_CA_df = ELEC_CA_df.rename(columns={"id":"ID", "state":"State", 
                                      "coordinates":"Lon & Lat", "fuel type":"Fuel Type"})
ELEC_CA_df.head()

,ID,State,Lon & Lat,Fuel Type
0,1517,CA,"[-118.3879713743439, 34.2483191527193]",ELEC
1,1519,CA,"[-118.448504, 34.052542]",ELEC
2,1523,CA,"[-118.271387, 34.040539]",ELEC
3,1525,CA,"[-118.248589, 34.059133]",ELEC
4,1531,CA,"[-118.096665, 33.759802]",ELEC


In [40]:
# Removing '[' from the beginning of 'Lat' values and ']' from the end of 'lon' values
for i, v in ELEC_CA_df.iterrows():
    print(v['Lon & Lat'][0], v['Lon & Lat'][1])
    if i == 5:
        break;

-118.3879713743439 34.2483191527193
-118.448504 34.052542
-118.271387 34.040539
-118.248589 34.059133
-118.096665 33.759802
-118.265628 33.770508


In [41]:
# Separating the "Lat & Lon" column into the two columns of "latitude" and "longitude"
ELEC_CA_df['Longitude'] = ELEC_CA_df['Lon & Lat'].map(lambda x: x[0])
ELEC_CA_df['Latitude'] = ELEC_CA_df['Lon & Lat'].map(lambda x: x[1])
ELEC_CA_df

,ID,State,Lon & Lat,Fuel Type,Longitude,Latitude
0,1517,CA,"[-118.3879713743439, 34.2483191527193]",ELEC,-118.387971,34.248319
1,1519,CA,"[-118.448504, 34.052542]",ELEC,-118.448504,34.052542
2,1523,CA,"[-118.271387, 34.040539]",ELEC,-118.271387,34.040539
3,1525,CA,"[-118.248589, 34.059133]",ELEC,-118.248589,34.059133
4,1531,CA,"[-118.096665, 33.759802]",ELEC,-118.096665,33.759802
...,...,...,...,...,...,...
16711,306367,CA,"[-116.969808, 33.209603]",ELEC,-116.969808,33.209603
16712,306368,CA,"[-116.969739, 33.209602]",ELEC,-116.969739,33.209602
16713,306408,CA,"[-119.28747, 36.32828]",ELEC,-119.287470,36.328280
16714,306414,CA,"[-116.969933, 33.20959]",ELEC,-116.969933,33.209590


In [42]:
# Deleting the "Lon & Lat" column
ELEC_CA_df = ELEC_CA_df.drop('Lon & Lat', axis=1)
ELEC_CA_df

,ID,State,Fuel Type,Longitude,Latitude
0,1517,CA,ELEC,-118.387971,34.248319
1,1519,CA,ELEC,-118.448504,34.052542
2,1523,CA,ELEC,-118.271387,34.040539
3,1525,CA,ELEC,-118.248589,34.059133
4,1531,CA,ELEC,-118.096665,33.759802
...,...,...,...,...,...
16711,306367,CA,ELEC,-116.969808,33.209603
16712,306368,CA,ELEC,-116.969739,33.209602
16713,306408,CA,ELEC,-119.287470,36.328280
16714,306414,CA,ELEC,-116.969933,33.209590


In [43]:
# Changing order of "Longitude" and "Latitude" columns to "Latitude" and "Longitude"
ELEC_CA_df = ELEC_CA_df.reindex(['ID','State','Fuel Type','Latitude', 'Longitude'], axis=1)
ELEC_CA_df

,ID,State,Fuel Type,Latitude,Longitude
0,1517,CA,ELEC,34.248319,-118.387971
1,1519,CA,ELEC,34.052542,-118.448504
2,1523,CA,ELEC,34.040539,-118.271387
3,1525,CA,ELEC,34.059133,-118.248589
4,1531,CA,ELEC,33.759802,-118.096665
...,...,...,...,...,...
16711,306367,CA,ELEC,33.209603,-116.969808
16712,306368,CA,ELEC,33.209602,-116.969739
16713,306408,CA,ELEC,36.328280,-119.287470
16714,306414,CA,ELEC,33.209590,-116.969933


In [44]:
# renaming "ELEC" to "Electric", "E85" to "Ethanol" and "LPG" to "Propane" in the "Fuel Type" column
ELEC_CA_df['Fuel Type'] = ELEC_CA_df['Fuel Type'].replace({'ELEC': 'Electric'})
ELEC_CA_df['Fuel Type'] = ELEC_CA_df['Fuel Type'].replace({'E85': 'Ethanol'})
ELEC_CA_df['Fuel Type'] = ELEC_CA_df['Fuel Type'].replace({'LPG': 'Propane'})
ELEC_CA_df

,ID,State,Fuel Type,Latitude,Longitude
0,1517,CA,Electric,34.248319,-118.387971
1,1519,CA,Electric,34.052542,-118.448504
2,1523,CA,Electric,34.040539,-118.271387
3,1525,CA,Electric,34.059133,-118.248589
4,1531,CA,Electric,33.759802,-118.096665
...,...,...,...,...,...
16711,306367,CA,Electric,33.209603,-116.969808
16712,306368,CA,Electric,33.209602,-116.969739
16713,306408,CA,Electric,36.328280,-119.287470
16714,306414,CA,Electric,33.209590,-116.969933


In [45]:
# renaming "CA" to "California", "TX" to "Texas" and "NY" to "New York" in the "State" column
ELEC_CA_df['State'] = ELEC_CA_df['State'].replace({'CA': 'California'})
ELEC_CA_df['State'] = ELEC_CA_df['State'].replace({'TX': 'Texas'})
ELEC_CA_df['State'] = ELEC_CA_df['State'].replace({'NY': 'New York'})
ELEC_CA_df

,ID,State,Fuel Type,Latitude,Longitude
0,1517,California,Electric,34.248319,-118.387971
1,1519,California,Electric,34.052542,-118.448504
2,1523,California,Electric,34.040539,-118.271387
3,1525,California,Electric,34.059133,-118.248589
4,1531,California,Electric,33.759802,-118.096665
...,...,...,...,...,...
16711,306367,California,Electric,33.209603,-116.969808
16712,306368,California,Electric,33.209602,-116.969739
16713,306408,California,Electric,36.328280,-119.287470
16714,306414,California,Electric,33.209590,-116.969933


### Exporting Transformed Dataframe to CSV

In [46]:
# Export the newly transformed dataframe into a csv
ELEC_CA_df.to_csv("../output_data/ETL/csv/ETL_ELEC_CA.csv", index_label="#")

# Performing ETL on "LPG_CA.csv" Dataset

## California Liquid Propane Gas Stations Data

In [47]:
# Create reference to CSV file
csv_path = Path("../output_data/data_analysis/csv/LPG_CA.csv")

# Import the CSV into a pandas DataFrame
LPG_CA_df = pd.read_csv(csv_path)
LPG_CA_df

,id,state,coordinates,fuel type
0,14744,CA,"[-117.7076, 33.615471]",LPG
1,14747,CA,"[-118.001197, 33.778977]",LPG
2,14755,CA,"[-119.2785736243286, 36.33042928891792]",LPG
3,14765,CA,"[-121.131276, 37.475136]",LPG
4,14777,CA,"[-122.048376, 38.00326]",LPG
...,...,...,...,...
252,213032,CA,"[-118.535701, 34.257609]",LPG
253,213992,CA,"[-117.21772, 33.164537]",LPG
254,236960,CA,"[-118.74337, 34.27132]",LPG
255,255568,CA,"[-118.28712, 33.842435]",LPG


In [48]:
# Get a brief summary of the LPG_CA_df DataFrame.
LPG_CA_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           257 non-null    int64 
 1   state        257 non-null    object
 2   coordinates  257 non-null    object
 3   fuel type    257 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.2+ KB


In [49]:
# converting column values to string
# source: https://docs.python.org/3/library/ast.html#ast.literal_eval
LPG_CA_df['coordinates'] = LPG_CA_df['coordinates'].map(lambda x: ast.literal_eval(x))

In [50]:
# Rename the id, state, coordinates and fuel type columns.
LPG_CA_df = LPG_CA_df.rename(columns={"id":"ID", "state":"State", 
                                      "coordinates":"Lon & Lat", "fuel type":"Fuel Type"})
LPG_CA_df.head()

,ID,State,Lon & Lat,Fuel Type
0,14744,CA,"[-117.7076, 33.615471]",LPG
1,14747,CA,"[-118.001197, 33.778977]",LPG
2,14755,CA,"[-119.2785736243286, 36.33042928891792]",LPG
3,14765,CA,"[-121.131276, 37.475136]",LPG
4,14777,CA,"[-122.048376, 38.00326]",LPG


In [51]:
# Removing '[' from the beginning of 'Lat' values and ']' from the end of 'lon' values
for i, v in LPG_CA_df.iterrows():
    print(v['Lon & Lat'][0], v['Lon & Lat'][1])
    if i == 5:
        break;

-117.7076 33.615471
-118.001197 33.778977
-119.2785736243286 36.33042928891792
-121.131276 37.475136
-122.048376 38.00326
-123.348474 39.392957


In [52]:
# Separating the "Lat & Lon" column into the two columns of "latitude" and "longitude"
LPG_CA_df['Longitude'] = LPG_CA_df['Lon & Lat'].map(lambda x: x[0])
LPG_CA_df['Latitude'] = LPG_CA_df['Lon & Lat'].map(lambda x: x[1])
LPG_CA_df

,ID,State,Lon & Lat,Fuel Type,Longitude,Latitude
0,14744,CA,"[-117.7076, 33.615471]",LPG,-117.707600,33.615471
1,14747,CA,"[-118.001197, 33.778977]",LPG,-118.001197,33.778977
2,14755,CA,"[-119.2785736243286, 36.33042928891792]",LPG,-119.278574,36.330429
3,14765,CA,"[-121.131276, 37.475136]",LPG,-121.131276,37.475136
4,14777,CA,"[-122.048376, 38.00326]",LPG,-122.048376,38.003260
...,...,...,...,...,...,...
252,213032,CA,"[-118.535701, 34.257609]",LPG,-118.535701,34.257609
253,213992,CA,"[-117.21772, 33.164537]",LPG,-117.217720,33.164537
254,236960,CA,"[-118.74337, 34.27132]",LPG,-118.743370,34.271320
255,255568,CA,"[-118.28712, 33.842435]",LPG,-118.287120,33.842435


In [53]:
# Deleting the "Lon & Lat" column
LPG_CA_df = LPG_CA_df.drop('Lon & Lat', axis=1)
LPG_CA_df

,ID,State,Fuel Type,Longitude,Latitude
0,14744,CA,LPG,-117.707600,33.615471
1,14747,CA,LPG,-118.001197,33.778977
2,14755,CA,LPG,-119.278574,36.330429
3,14765,CA,LPG,-121.131276,37.475136
4,14777,CA,LPG,-122.048376,38.003260
...,...,...,...,...,...
252,213032,CA,LPG,-118.535701,34.257609
253,213992,CA,LPG,-117.217720,33.164537
254,236960,CA,LPG,-118.743370,34.271320
255,255568,CA,LPG,-118.287120,33.842435


In [54]:
# Changing order of "Longitude" and "Latitude" columns to "Latitude" and "Longitude"
LPG_CA_df = LPG_CA_df.reindex(['ID','State','Fuel Type','Latitude', 'Longitude'], axis=1)
LPG_CA_df

,ID,State,Fuel Type,Latitude,Longitude
0,14744,CA,LPG,33.615471,-117.707600
1,14747,CA,LPG,33.778977,-118.001197
2,14755,CA,LPG,36.330429,-119.278574
3,14765,CA,LPG,37.475136,-121.131276
4,14777,CA,LPG,38.003260,-122.048376
...,...,...,...,...,...
252,213032,CA,LPG,34.257609,-118.535701
253,213992,CA,LPG,33.164537,-117.217720
254,236960,CA,LPG,34.271320,-118.743370
255,255568,CA,LPG,33.842435,-118.287120


In [55]:
# renaming "ELEC" to "Electric", "E85" to "Ethanol" and "LPG" to "Propane" in the "Fuel Type" column
LPG_CA_df['Fuel Type'] = LPG_CA_df['Fuel Type'].replace({'ELEC': 'Electric'})
LPG_CA_df['Fuel Type'] = LPG_CA_df['Fuel Type'].replace({'E85': 'Ethanol'})
LPG_CA_df['Fuel Type'] = LPG_CA_df['Fuel Type'].replace({'LPG': 'Propane'})
LPG_CA_df

,ID,State,Fuel Type,Latitude,Longitude
0,14744,CA,Propane,33.615471,-117.707600
1,14747,CA,Propane,33.778977,-118.001197
2,14755,CA,Propane,36.330429,-119.278574
3,14765,CA,Propane,37.475136,-121.131276
4,14777,CA,Propane,38.003260,-122.048376
...,...,...,...,...,...
252,213032,CA,Propane,34.257609,-118.535701
253,213992,CA,Propane,33.164537,-117.217720
254,236960,CA,Propane,34.271320,-118.743370
255,255568,CA,Propane,33.842435,-118.287120


In [56]:
# renaming "CA" to "California", "TX" to "Texas" and "NY" to "New York" in the "State" column
LPG_CA_df['State'] = LPG_CA_df['State'].replace({'CA': 'California'})
LPG_CA_df['State'] = LPG_CA_df['State'].replace({'TX': 'Texas'})
LPG_CA_df['State'] = LPG_CA_df['State'].replace({'NY': 'New York'})
LPG_CA_df

,ID,State,Fuel Type,Latitude,Longitude
0,14744,California,Propane,33.615471,-117.707600
1,14747,California,Propane,33.778977,-118.001197
2,14755,California,Propane,36.330429,-119.278574
3,14765,California,Propane,37.475136,-121.131276
4,14777,California,Propane,38.003260,-122.048376
...,...,...,...,...,...
252,213032,California,Propane,34.257609,-118.535701
253,213992,California,Propane,33.164537,-117.217720
254,236960,California,Propane,34.271320,-118.743370
255,255568,California,Propane,33.842435,-118.287120


### Exporting Transformed Dataframe to CSV

In [57]:
# Export the newly transformed dataframe into a csv
LPG_CA_df.to_csv("../output_data/ETL/csv/ETL_LPG_CA.csv", index_label="#")

# Performing ETL on "top3_alt_fuels_TX.csv" Dataset

## Top 3 Alternative Fuel Stations Data for Texas

In [58]:
# Create reference to CSV file
csv_path = Path("../output_data/data_analysis/csv/top3_alt_fuels_TX.csv")

# Import the CSV into a pandas DataFrame
top3_alt_fuels_TX_df = pd.read_csv(csv_path)
top3_alt_fuels_TX_df

,id,state,coordinates,fuel type
0,13298,TX,"[-101.885391, 35.267353]",LPG
1,14387,TX,"[-96.73629, 32.684726]",LPG
2,14389,TX,"[-95.54979990870663, 33.72109484294474]",LPG
3,14392,TX,"[-94.226198, 33.095005]",LPG
4,14399,TX,"[-95.345249, 32.182469]",LPG
...,...,...,...,...
3651,306460,TX,"[-95.98519, 31.25954]",ELEC
3652,306461,TX,"[-95.98519, 31.25951]",ELEC
3653,306462,TX,"[-95.98519, 31.25942]",ELEC
3654,306463,TX,"[-95.985225, 31.25934]",ELEC


In [59]:
# Get a brief summary of the top3_alt_fuels_TX_df DataFrame.
top3_alt_fuels_TX_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3656 entries, 0 to 3655
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           3656 non-null   int64 
 1   state        3656 non-null   object
 2   coordinates  3656 non-null   object
 3   fuel type    3656 non-null   object
dtypes: int64(1), object(3)
memory usage: 114.4+ KB


In [60]:
# converting column values to string
# source: https://docs.python.org/3/library/ast.html#ast.literal_eval
top3_alt_fuels_TX_df['coordinates'] = top3_alt_fuels_TX_df['coordinates'].map(lambda x: ast.literal_eval(x))

In [61]:
# Rename the id, state, coordinates and fuel type columns.
top3_alt_fuels_TX_df = top3_alt_fuels_TX_df.rename(columns={"id":"ID", "state":"State", 
                                                            "coordinates":"Lon & Lat", "fuel type":"Fuel Type"})
top3_alt_fuels_TX_df.head()

,ID,State,Lon & Lat,Fuel Type
0,13298,TX,"[-101.885391, 35.267353]",LPG
1,14387,TX,"[-96.73629, 32.684726]",LPG
2,14389,TX,"[-95.54979990870663, 33.72109484294474]",LPG
3,14392,TX,"[-94.226198, 33.095005]",LPG
4,14399,TX,"[-95.345249, 32.182469]",LPG


In [62]:
# Removing '[' from the beginning of 'Lat' values and ']' from the end of 'lon' values
for i, v in top3_alt_fuels_TX_df.iterrows():
    print(v['Lon & Lat'][0], v['Lon & Lat'][1])
    if i == 5:
        break;

-101.885391 35.267353
-96.73629 32.684726
-95.54979990870663 33.72109484294474
-94.226198 33.095005
-95.345249 32.182469
-95.719888 31.702799


In [63]:
# Separating the "Lat & Lon" column into the two columns of "latitude" and "longitude"
top3_alt_fuels_TX_df['Longitude'] = top3_alt_fuels_TX_df['Lon & Lat'].map(lambda x: x[0])
top3_alt_fuels_TX_df['Latitude'] = top3_alt_fuels_TX_df['Lon & Lat'].map(lambda x: x[1])
top3_alt_fuels_TX_df

,ID,State,Lon & Lat,Fuel Type,Longitude,Latitude
0,13298,TX,"[-101.885391, 35.267353]",LPG,-101.885391,35.267353
1,14387,TX,"[-96.73629, 32.684726]",LPG,-96.736290,32.684726
2,14389,TX,"[-95.54979990870663, 33.72109484294474]",LPG,-95.549800,33.721095
3,14392,TX,"[-94.226198, 33.095005]",LPG,-94.226198,33.095005
4,14399,TX,"[-95.345249, 32.182469]",LPG,-95.345249,32.182469
...,...,...,...,...,...,...
3651,306460,TX,"[-95.98519, 31.25954]",ELEC,-95.985190,31.259540
3652,306461,TX,"[-95.98519, 31.25951]",ELEC,-95.985190,31.259510
3653,306462,TX,"[-95.98519, 31.25942]",ELEC,-95.985190,31.259420
3654,306463,TX,"[-95.985225, 31.25934]",ELEC,-95.985225,31.259340


In [64]:
# Deleting the "Lon & Lat" column
top3_alt_fuels_TX_df = top3_alt_fuels_TX_df.drop('Lon & Lat', axis=1)
top3_alt_fuels_TX_df

,ID,State,Fuel Type,Longitude,Latitude
0,13298,TX,LPG,-101.885391,35.267353
1,14387,TX,LPG,-96.736290,32.684726
2,14389,TX,LPG,-95.549800,33.721095
3,14392,TX,LPG,-94.226198,33.095005
4,14399,TX,LPG,-95.345249,32.182469
...,...,...,...,...,...
3651,306460,TX,ELEC,-95.985190,31.259540
3652,306461,TX,ELEC,-95.985190,31.259510
3653,306462,TX,ELEC,-95.985190,31.259420
3654,306463,TX,ELEC,-95.985225,31.259340


In [65]:
# Changing order of "Longitude" and "Latitude" columns to "Latitude" and "Longitude"
top3_alt_fuels_TX_df = top3_alt_fuels_TX_df.reindex(['ID','State','Fuel Type','Latitude', 'Longitude'], axis=1)
top3_alt_fuels_TX_df

,ID,State,Fuel Type,Latitude,Longitude
0,13298,TX,LPG,35.267353,-101.885391
1,14387,TX,LPG,32.684726,-96.736290
2,14389,TX,LPG,33.721095,-95.549800
3,14392,TX,LPG,33.095005,-94.226198
4,14399,TX,LPG,32.182469,-95.345249
...,...,...,...,...,...
3651,306460,TX,ELEC,31.259540,-95.985190
3652,306461,TX,ELEC,31.259510,-95.985190
3653,306462,TX,ELEC,31.259420,-95.985190
3654,306463,TX,ELEC,31.259340,-95.985225


In [66]:
# renaming "ELEC" to "Electric", "E85" to "Ethanol" and "LPG" to "Propane" in the "Fuel Type" column
top3_alt_fuels_TX_df['Fuel Type'] = top3_alt_fuels_TX_df['Fuel Type'].replace({'ELEC': 'Electric'})
top3_alt_fuels_TX_df['Fuel Type'] = top3_alt_fuels_TX_df['Fuel Type'].replace({'E85': 'Ethanol'})
top3_alt_fuels_TX_df['Fuel Type'] = top3_alt_fuels_TX_df['Fuel Type'].replace({'LPG': 'Propane'})
top3_alt_fuels_TX_df

,ID,State,Fuel Type,Latitude,Longitude
0,13298,TX,Propane,35.267353,-101.885391
1,14387,TX,Propane,32.684726,-96.736290
2,14389,TX,Propane,33.721095,-95.549800
3,14392,TX,Propane,33.095005,-94.226198
4,14399,TX,Propane,32.182469,-95.345249
...,...,...,...,...,...
3651,306460,TX,Electric,31.259540,-95.985190
3652,306461,TX,Electric,31.259510,-95.985190
3653,306462,TX,Electric,31.259420,-95.985190
3654,306463,TX,Electric,31.259340,-95.985225


In [67]:
# renaming "CA" to "California", "TX" to "Texas" and "NY" to "New York" in the "State" column
top3_alt_fuels_TX_df['State'] = top3_alt_fuels_TX_df['State'].replace({'CA': 'California'})
top3_alt_fuels_TX_df['State'] = top3_alt_fuels_TX_df['State'].replace({'TX': 'Texas'})
top3_alt_fuels_TX_df['State'] = top3_alt_fuels_TX_df['State'].replace({'NY': 'New York'})
top3_alt_fuels_TX_df

,ID,State,Fuel Type,Latitude,Longitude
0,13298,Texas,Propane,35.267353,-101.885391
1,14387,Texas,Propane,32.684726,-96.736290
2,14389,Texas,Propane,33.721095,-95.549800
3,14392,Texas,Propane,33.095005,-94.226198
4,14399,Texas,Propane,32.182469,-95.345249
...,...,...,...,...,...
3651,306460,Texas,Electric,31.259540,-95.985190
3652,306461,Texas,Electric,31.259510,-95.985190
3653,306462,Texas,Electric,31.259420,-95.985190
3654,306463,Texas,Electric,31.259340,-95.985225


### Exporting Transformed Dataframe to CSV

In [68]:
# Export the newly transformed dataframe into a csv
top3_alt_fuels_TX_df.to_csv("../output_data/ETL/csv/ETL_top3_alt_fuels_TX.csv", index_label="#")

# Performing ETL on "E85_TX.csv" Dataset

## Texas Ethanol Fuel Stations Data

In [69]:
# Create reference to CSV file
csv_path = Path("../output_data/data_analysis/csv/E85_TX.csv")

# Import the CSV into a pandas DataFrame
E85_TX_df = pd.read_csv(csv_path)
E85_TX_df

,id,state,coordinates,fuel type
0,29386,TX,"[-97.671997, 30.405517]",E85
1,29387,TX,"[-97.8206, 30.08711]",E85
2,29388,TX,"[-95.631595, 29.901435]",E85
3,29389,TX,"[-96.960994, 32.908539]",E85
4,29390,TX,"[-97.736095, 31.09186]",E85
...,...,...,...,...
259,251747,TX,"[-97.10904, 32.85226]",E85
260,252792,TX,"[-96.30052160713196, 30.55925326796013]",E85
261,260069,TX,"[-94.97697, 29.75917]",E85
262,262251,TX,"[-94.08458, 33.4544]",E85


In [70]:
# Get a brief summary of the E85_TX_df DataFrame.
E85_TX_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           264 non-null    int64 
 1   state        264 non-null    object
 2   coordinates  264 non-null    object
 3   fuel type    264 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


In [71]:
# converting column values to string
# source: https://docs.python.org/3/library/ast.html#ast.literal_eval
E85_TX_df['coordinates'] = E85_TX_df['coordinates'].map(lambda x: ast.literal_eval(x))

In [72]:
# Rename the id, state, coordinates and fuel type columns.
E85_TX_df = E85_TX_df.rename(columns={"id":"ID", "state":"State", 
                                      "coordinates":"Lon & Lat", "fuel type":"Fuel Type"})
E85_TX_df.head()

,ID,State,Lon & Lat,Fuel Type
0,29386,TX,"[-97.671997, 30.405517]",E85
1,29387,TX,"[-97.8206, 30.08711]",E85
2,29388,TX,"[-95.631595, 29.901435]",E85
3,29389,TX,"[-96.960994, 32.908539]",E85
4,29390,TX,"[-97.736095, 31.09186]",E85


In [73]:
# Removing '[' from the beginning of 'Lat' values and ']' from the end of 'lon' values
for i, v in E85_TX_df.iterrows():
    print(v['Lon & Lat'][0], v['Lon & Lat'][1])
    if i == 5:
        break;

-97.671997 30.405517
-97.8206 30.08711
-95.631595 29.901435
-96.960994 32.908539
-97.736095 31.09186
-98.2779 29.59841


In [74]:
# Separating the "Lat & Lon" column into the two columns of "latitude" and "longitude"
E85_TX_df['Longitude'] = E85_TX_df['Lon & Lat'].map(lambda x: x[0])
E85_TX_df['Latitude'] = E85_TX_df['Lon & Lat'].map(lambda x: x[1])
E85_TX_df

,ID,State,Lon & Lat,Fuel Type,Longitude,Latitude
0,29386,TX,"[-97.671997, 30.405517]",E85,-97.671997,30.405517
1,29387,TX,"[-97.8206, 30.08711]",E85,-97.820600,30.087110
2,29388,TX,"[-95.631595, 29.901435]",E85,-95.631595,29.901435
3,29389,TX,"[-96.960994, 32.908539]",E85,-96.960994,32.908539
4,29390,TX,"[-97.736095, 31.09186]",E85,-97.736095,31.091860
...,...,...,...,...,...,...
259,251747,TX,"[-97.10904, 32.85226]",E85,-97.109040,32.852260
260,252792,TX,"[-96.30052160713196, 30.55925326796013]",E85,-96.300522,30.559253
261,260069,TX,"[-94.97697, 29.75917]",E85,-94.976970,29.759170
262,262251,TX,"[-94.08458, 33.4544]",E85,-94.084580,33.454400


In [75]:
# Deleting the "Lon & Lat" column
E85_TX_df = E85_TX_df.drop('Lon & Lat', axis=1)
E85_TX_df

,ID,State,Fuel Type,Longitude,Latitude
0,29386,TX,E85,-97.671997,30.405517
1,29387,TX,E85,-97.820600,30.087110
2,29388,TX,E85,-95.631595,29.901435
3,29389,TX,E85,-96.960994,32.908539
4,29390,TX,E85,-97.736095,31.091860
...,...,...,...,...,...
259,251747,TX,E85,-97.109040,32.852260
260,252792,TX,E85,-96.300522,30.559253
261,260069,TX,E85,-94.976970,29.759170
262,262251,TX,E85,-94.084580,33.454400


In [76]:
# Changing order of "Longitude" and "Latitude" columns to "Latitude" and "Longitude"
E85_TX_df = E85_TX_df.reindex(['ID','State','Fuel Type','Latitude', 'Longitude'], axis=1)
E85_TX_df

,ID,State,Fuel Type,Latitude,Longitude
0,29386,TX,E85,30.405517,-97.671997
1,29387,TX,E85,30.087110,-97.820600
2,29388,TX,E85,29.901435,-95.631595
3,29389,TX,E85,32.908539,-96.960994
4,29390,TX,E85,31.091860,-97.736095
...,...,...,...,...,...
259,251747,TX,E85,32.852260,-97.109040
260,252792,TX,E85,30.559253,-96.300522
261,260069,TX,E85,29.759170,-94.976970
262,262251,TX,E85,33.454400,-94.084580


In [77]:
# renaming "ELEC" to "Electric", "E85" to "Ethanol" and "LPG" to "Propane" in the "Fuel Type" column
E85_TX_df['Fuel Type'] = E85_TX_df['Fuel Type'].replace({'ELEC': 'Electric'})
E85_TX_df['Fuel Type'] = E85_TX_df['Fuel Type'].replace({'E85': 'Ethanol'})
E85_TX_df['Fuel Type'] = E85_TX_df['Fuel Type'].replace({'LPG': 'Propane'})
E85_TX_df

,ID,State,Fuel Type,Latitude,Longitude
0,29386,TX,Ethanol,30.405517,-97.671997
1,29387,TX,Ethanol,30.087110,-97.820600
2,29388,TX,Ethanol,29.901435,-95.631595
3,29389,TX,Ethanol,32.908539,-96.960994
4,29390,TX,Ethanol,31.091860,-97.736095
...,...,...,...,...,...
259,251747,TX,Ethanol,32.852260,-97.109040
260,252792,TX,Ethanol,30.559253,-96.300522
261,260069,TX,Ethanol,29.759170,-94.976970
262,262251,TX,Ethanol,33.454400,-94.084580


In [78]:
# renaming "CA" to "California", "TX" to "Texas" and "NY" to "New York" in the "State" column
E85_TX_df['State'] = E85_TX_df['State'].replace({'CA': 'California'})
E85_TX_df['State'] = E85_TX_df['State'].replace({'TX': 'Texas'})
E85_TX_df['State'] = E85_TX_df['State'].replace({'NY': 'New York'})
E85_TX_df

,ID,State,Fuel Type,Latitude,Longitude
0,29386,Texas,Ethanol,30.405517,-97.671997
1,29387,Texas,Ethanol,30.087110,-97.820600
2,29388,Texas,Ethanol,29.901435,-95.631595
3,29389,Texas,Ethanol,32.908539,-96.960994
4,29390,Texas,Ethanol,31.091860,-97.736095
...,...,...,...,...,...
259,251747,Texas,Ethanol,32.852260,-97.109040
260,252792,Texas,Ethanol,30.559253,-96.300522
261,260069,Texas,Ethanol,29.759170,-94.976970
262,262251,Texas,Ethanol,33.454400,-94.084580


### Exporting Transformed Dataframe to CSV

In [79]:
# Export the newly transformed dataframe into a csv
E85_TX_df.to_csv("../output_data/ETL/csv/ETL_E85_TX.csv", index_label="#")

# Performing ETL on "ELEC_TX.csv" Dataset

## Texas Electric Car Recharging Stations Data

In [80]:
# Create reference to CSV file
csv_path = Path("../output_data/data_analysis/csv/ELEC_TX.csv")

# Import the CSV into a pandas DataFrame
ELEC_TX_df = pd.read_csv(csv_path)
ELEC_TX_df

,id,state,coordinates,fuel type
0,36378,TX,"[-95.372854, 29.756604]",ELEC
1,39114,TX,"[-97.759109, 30.206145]",ELEC
2,39708,TX,"[-99.771463, 32.408909]",ELEC
3,39709,TX,"[-101.938937, 35.159354]",ELEC
4,39710,TX,"[-97.087169, 32.676458]",ELEC
...,...,...,...,...
3029,306460,TX,"[-95.98519, 31.25954]",ELEC
3030,306461,TX,"[-95.98519, 31.25951]",ELEC
3031,306462,TX,"[-95.98519, 31.25942]",ELEC
3032,306463,TX,"[-95.985225, 31.25934]",ELEC


In [81]:
# Get a brief summary of the ELEC_TX_df DataFrame.
ELEC_TX_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3034 entries, 0 to 3033
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           3034 non-null   int64 
 1   state        3034 non-null   object
 2   coordinates  3034 non-null   object
 3   fuel type    3034 non-null   object
dtypes: int64(1), object(3)
memory usage: 94.9+ KB


In [82]:
# converting column values to string
# source: https://docs.python.org/3/library/ast.html#ast.literal_eval
ELEC_TX_df['coordinates'] = ELEC_TX_df['coordinates'].map(lambda x: ast.literal_eval(x))

In [83]:
# Rename the id, state, coordinates and fuel type columns.
ELEC_TX_df = ELEC_TX_df.rename(columns={"id":"ID", "state":"State", 
                                      "coordinates":"Lon & Lat", "fuel type":"Fuel Type"})
ELEC_TX_df.head()

,ID,State,Lon & Lat,Fuel Type
0,36378,TX,"[-95.372854, 29.756604]",ELEC
1,39114,TX,"[-97.759109, 30.206145]",ELEC
2,39708,TX,"[-99.771463, 32.408909]",ELEC
3,39709,TX,"[-101.938937, 35.159354]",ELEC
4,39710,TX,"[-97.087169, 32.676458]",ELEC


In [84]:
# Removing '[' from the beginning of 'Lat' values and ']' from the end of 'lon' values
for i, v in ELEC_TX_df.iterrows():
    print(v['Lon & Lat'][0], v['Lon & Lat'][1])
    if i == 5:
        break;

-95.372854 29.756604
-97.759109 30.206145
-99.771463 32.408909
-101.938937 35.159354
-97.087169 32.676458
-97.731445 30.374542


In [85]:
# Separating the "Lat & Lon" column into the two columns of "latitude" and "longitude"
ELEC_TX_df['Longitude'] = ELEC_TX_df['Lon & Lat'].map(lambda x: x[0])
ELEC_TX_df['Latitude'] = ELEC_TX_df['Lon & Lat'].map(lambda x: x[1])
ELEC_TX_df

,ID,State,Lon & Lat,Fuel Type,Longitude,Latitude
0,36378,TX,"[-95.372854, 29.756604]",ELEC,-95.372854,29.756604
1,39114,TX,"[-97.759109, 30.206145]",ELEC,-97.759109,30.206145
2,39708,TX,"[-99.771463, 32.408909]",ELEC,-99.771463,32.408909
3,39709,TX,"[-101.938937, 35.159354]",ELEC,-101.938937,35.159354
4,39710,TX,"[-97.087169, 32.676458]",ELEC,-97.087169,32.676458
...,...,...,...,...,...,...
3029,306460,TX,"[-95.98519, 31.25954]",ELEC,-95.985190,31.259540
3030,306461,TX,"[-95.98519, 31.25951]",ELEC,-95.985190,31.259510
3031,306462,TX,"[-95.98519, 31.25942]",ELEC,-95.985190,31.259420
3032,306463,TX,"[-95.985225, 31.25934]",ELEC,-95.985225,31.259340


In [86]:
# Deleting the "Lon & Lat" column
ELEC_TX_df = ELEC_TX_df.drop('Lon & Lat', axis=1)
ELEC_TX_df

,ID,State,Fuel Type,Longitude,Latitude
0,36378,TX,ELEC,-95.372854,29.756604
1,39114,TX,ELEC,-97.759109,30.206145
2,39708,TX,ELEC,-99.771463,32.408909
3,39709,TX,ELEC,-101.938937,35.159354
4,39710,TX,ELEC,-97.087169,32.676458
...,...,...,...,...,...
3029,306460,TX,ELEC,-95.985190,31.259540
3030,306461,TX,ELEC,-95.985190,31.259510
3031,306462,TX,ELEC,-95.985190,31.259420
3032,306463,TX,ELEC,-95.985225,31.259340


In [87]:
# Changing order of "Longitude" and "Latitude" columns to "Latitude" and "Longitude"
ELEC_TX_df = ELEC_TX_df.reindex(['ID','State','Fuel Type','Latitude', 'Longitude'], axis=1)
ELEC_TX_df

,ID,State,Fuel Type,Latitude,Longitude
0,36378,TX,ELEC,29.756604,-95.372854
1,39114,TX,ELEC,30.206145,-97.759109
2,39708,TX,ELEC,32.408909,-99.771463
3,39709,TX,ELEC,35.159354,-101.938937
4,39710,TX,ELEC,32.676458,-97.087169
...,...,...,...,...,...
3029,306460,TX,ELEC,31.259540,-95.985190
3030,306461,TX,ELEC,31.259510,-95.985190
3031,306462,TX,ELEC,31.259420,-95.985190
3032,306463,TX,ELEC,31.259340,-95.985225


In [88]:
# renaming "ELEC" to "Electric", "E85" to "Ethanol" and "LPG" to "Propane" in the "Fuel Type" column
ELEC_TX_df['Fuel Type'] = ELEC_TX_df['Fuel Type'].replace({'ELEC': 'Electric'})
ELEC_TX_df['Fuel Type'] = ELEC_TX_df['Fuel Type'].replace({'E85': 'Ethanol'})
ELEC_TX_df['Fuel Type'] = ELEC_TX_df['Fuel Type'].replace({'LPG': 'Propane'})
ELEC_TX_df

,ID,State,Fuel Type,Latitude,Longitude
0,36378,TX,Electric,29.756604,-95.372854
1,39114,TX,Electric,30.206145,-97.759109
2,39708,TX,Electric,32.408909,-99.771463
3,39709,TX,Electric,35.159354,-101.938937
4,39710,TX,Electric,32.676458,-97.087169
...,...,...,...,...,...
3029,306460,TX,Electric,31.259540,-95.985190
3030,306461,TX,Electric,31.259510,-95.985190
3031,306462,TX,Electric,31.259420,-95.985190
3032,306463,TX,Electric,31.259340,-95.985225


In [89]:
# renaming "CA" to "California", "TX" to "Texas" and "NY" to "New York" in the "State" column
ELEC_TX_df['State'] = ELEC_TX_df['State'].replace({'CA': 'California'})
ELEC_TX_df['State'] = ELEC_TX_df['State'].replace({'TX': 'Texas'})
ELEC_TX_df['State'] = ELEC_TX_df['State'].replace({'NY': 'New York'})
ELEC_TX_df

,ID,State,Fuel Type,Latitude,Longitude
0,36378,Texas,Electric,29.756604,-95.372854
1,39114,Texas,Electric,30.206145,-97.759109
2,39708,Texas,Electric,32.408909,-99.771463
3,39709,Texas,Electric,35.159354,-101.938937
4,39710,Texas,Electric,32.676458,-97.087169
...,...,...,...,...,...
3029,306460,Texas,Electric,31.259540,-95.985190
3030,306461,Texas,Electric,31.259510,-95.985190
3031,306462,Texas,Electric,31.259420,-95.985190
3032,306463,Texas,Electric,31.259340,-95.985225


### Exporting Transformed Dataframe to CSV

In [90]:
# Export the newly transformed dataframe into a csv
ELEC_TX_df.to_csv("../output_data/ETL/csv/ETL_ELEC_TX.csv", index_label="#")

# Performing ETL on "LPG_TX.csv" Dataset

## Texas Liquid Propane Gas Stations Data

In [91]:
# Create reference to CSV file
csv_path = Path("../output_data/data_analysis/csv/LPG_TX.csv")

# Import the CSV into a pandas DataFrame
LPG_TX_df = pd.read_csv(csv_path)
LPG_TX_df

,id,state,coordinates,fuel type
0,13298,TX,"[-101.885391, 35.267353]",LPG
1,14387,TX,"[-96.73629, 32.684726]",LPG
2,14389,TX,"[-95.54979990870663, 33.72109484294474]",LPG
3,14392,TX,"[-94.226198, 33.095005]",LPG
4,14399,TX,"[-95.345249, 32.182469]",LPG
...,...,...,...,...
353,168508,TX,"[-95.240462, 32.26469]",LPG
354,187542,TX,"[-97.287105, 32.679451]",LPG
355,188086,TX,"[-94.192517, 33.107791]",LPG
356,251745,TX,"[-97.67691, 26.20453]",LPG


In [92]:
# Get a brief summary of the LPG_TX_df DataFrame.
LPG_TX_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358 entries, 0 to 357
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           358 non-null    int64 
 1   state        358 non-null    object
 2   coordinates  358 non-null    object
 3   fuel type    358 non-null    object
dtypes: int64(1), object(3)
memory usage: 11.3+ KB


In [93]:
# converting column values to string
# source: https://docs.python.org/3/library/ast.html#ast.literal_eval
LPG_TX_df['coordinates'] = LPG_TX_df['coordinates'].map(lambda x: ast.literal_eval(x))

In [94]:
# Rename the id, state, coordinates and fuel type columns.
LPG_TX_df = LPG_TX_df.rename(columns={"id":"ID", "state":"State", 
                                      "coordinates":"Lon & Lat", "fuel type":"Fuel Type"})
LPG_TX_df.head()

,ID,State,Lon & Lat,Fuel Type
0,13298,TX,"[-101.885391, 35.267353]",LPG
1,14387,TX,"[-96.73629, 32.684726]",LPG
2,14389,TX,"[-95.54979990870663, 33.72109484294474]",LPG
3,14392,TX,"[-94.226198, 33.095005]",LPG
4,14399,TX,"[-95.345249, 32.182469]",LPG


In [95]:
# Removing '[' from the beginning of 'Lat' values and ']' from the end of 'lon' values
for i, v in LPG_TX_df.iterrows():
    print(v['Lon & Lat'][0], v['Lon & Lat'][1])
    if i == 5:
        break;

-101.885391 35.267353
-96.73629 32.684726
-95.54979990870663 33.72109484294474
-94.226198 33.095005
-95.345249 32.182469
-95.719888 31.702799


In [96]:
# Separating the "Lat & Lon" column into the two columns of "latitude" and "longitude"
LPG_TX_df['Longitude'] = LPG_TX_df['Lon & Lat'].map(lambda x: x[0])
LPG_TX_df['Latitude'] = LPG_TX_df['Lon & Lat'].map(lambda x: x[1])
LPG_TX_df

,ID,State,Lon & Lat,Fuel Type,Longitude,Latitude
0,13298,TX,"[-101.885391, 35.267353]",LPG,-101.885391,35.267353
1,14387,TX,"[-96.73629, 32.684726]",LPG,-96.736290,32.684726
2,14389,TX,"[-95.54979990870663, 33.72109484294474]",LPG,-95.549800,33.721095
3,14392,TX,"[-94.226198, 33.095005]",LPG,-94.226198,33.095005
4,14399,TX,"[-95.345249, 32.182469]",LPG,-95.345249,32.182469
...,...,...,...,...,...,...
353,168508,TX,"[-95.240462, 32.26469]",LPG,-95.240462,32.264690
354,187542,TX,"[-97.287105, 32.679451]",LPG,-97.287105,32.679451
355,188086,TX,"[-94.192517, 33.107791]",LPG,-94.192517,33.107791
356,251745,TX,"[-97.67691, 26.20453]",LPG,-97.676910,26.204530


In [97]:
# Deleting the "Lon & Lat" column
LPG_TX_df = LPG_TX_df.drop('Lon & Lat', axis=1)
LPG_TX_df

,ID,State,Fuel Type,Longitude,Latitude
0,13298,TX,LPG,-101.885391,35.267353
1,14387,TX,LPG,-96.736290,32.684726
2,14389,TX,LPG,-95.549800,33.721095
3,14392,TX,LPG,-94.226198,33.095005
4,14399,TX,LPG,-95.345249,32.182469
...,...,...,...,...,...
353,168508,TX,LPG,-95.240462,32.264690
354,187542,TX,LPG,-97.287105,32.679451
355,188086,TX,LPG,-94.192517,33.107791
356,251745,TX,LPG,-97.676910,26.204530


In [98]:
# Changing order of "Longitude" and "Latitude" columns to "Latitude" and "Longitude"
LPG_TX_df = LPG_TX_df.reindex(['ID','State','Fuel Type','Latitude', 'Longitude'], axis=1)
LPG_TX_df

,ID,State,Fuel Type,Latitude,Longitude
0,13298,TX,LPG,35.267353,-101.885391
1,14387,TX,LPG,32.684726,-96.736290
2,14389,TX,LPG,33.721095,-95.549800
3,14392,TX,LPG,33.095005,-94.226198
4,14399,TX,LPG,32.182469,-95.345249
...,...,...,...,...,...
353,168508,TX,LPG,32.264690,-95.240462
354,187542,TX,LPG,32.679451,-97.287105
355,188086,TX,LPG,33.107791,-94.192517
356,251745,TX,LPG,26.204530,-97.676910


In [99]:
# renaming "ELEC" to "Electric", "E85" to "Ethanol" and "LPG" to "Propane" in the "Fuel Type" column
LPG_TX_df['Fuel Type'] = LPG_TX_df['Fuel Type'].replace({'ELEC': 'Electric'})
LPG_TX_df['Fuel Type'] = LPG_TX_df['Fuel Type'].replace({'E85': 'Ethanol'})
LPG_TX_df['Fuel Type'] = LPG_TX_df['Fuel Type'].replace({'LPG': 'Propane'})
LPG_TX_df

,ID,State,Fuel Type,Latitude,Longitude
0,13298,TX,Propane,35.267353,-101.885391
1,14387,TX,Propane,32.684726,-96.736290
2,14389,TX,Propane,33.721095,-95.549800
3,14392,TX,Propane,33.095005,-94.226198
4,14399,TX,Propane,32.182469,-95.345249
...,...,...,...,...,...
353,168508,TX,Propane,32.264690,-95.240462
354,187542,TX,Propane,32.679451,-97.287105
355,188086,TX,Propane,33.107791,-94.192517
356,251745,TX,Propane,26.204530,-97.676910


In [100]:
# renaming "CA" to "California", "TX" to "Texas" and "NY" to "New York" in the "State" column
LPG_TX_df['State'] = LPG_TX_df['State'].replace({'CA': 'California'})
LPG_TX_df['State'] = LPG_TX_df['State'].replace({'TX': 'Texas'})
LPG_TX_df['State'] = LPG_TX_df['State'].replace({'NY': 'New York'})
LPG_TX_df

,ID,State,Fuel Type,Latitude,Longitude
0,13298,Texas,Propane,35.267353,-101.885391
1,14387,Texas,Propane,32.684726,-96.736290
2,14389,Texas,Propane,33.721095,-95.549800
3,14392,Texas,Propane,33.095005,-94.226198
4,14399,Texas,Propane,32.182469,-95.345249
...,...,...,...,...,...
353,168508,Texas,Propane,32.264690,-95.240462
354,187542,Texas,Propane,32.679451,-97.287105
355,188086,Texas,Propane,33.107791,-94.192517
356,251745,Texas,Propane,26.204530,-97.676910


### Exporting Transformed Dataframe to CSV

In [101]:
# Export the newly transformed dataframe into a csv
LPG_TX_df.to_csv("../output_data/ETL/csv/ETL_LPG_TX.csv", index_label="#")

# Performing ETL on "top3_alt_fuels_NY.csv" Dataset

## Top 3 Alternative Fuel Stations Data for New York

In [102]:
# Create reference to CSV file
csv_path = Path("../output_data/data_analysis/csv/top3_alt_fuels_NY.csv")

# Import the CSV into a pandas DataFrame
top3_alt_fuels_NY_df = pd.read_csv(csv_path)
top3_alt_fuels_NY_df

,id,state,coordinates,fuel type
0,13401,NY,"[-74.672938, 41.651953]",LPG
1,13444,NY,"[-78.85922572883605, 42.8601082048398]",LPG
2,19234,NY,"[-73.634055, 40.658112]",LPG
3,19236,NY,"[-73.160001, 40.851029]",LPG
4,33001,NY,"[-73.464669, 40.692812]",E85
...,...,...,...,...
4138,306301,NY,"[-73.15867, 40.86042]",ELEC
4139,306302,NY,"[-73.15845, 40.860767]",ELEC
4140,306335,NY,"[-72.755549, 40.956315]",ELEC
4141,306336,NY,"[-72.755536, 40.956268]",ELEC


In [103]:
# Get a brief summary of the top3_alt_fuels_NY_df DataFrame.
top3_alt_fuels_NY_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4143 entries, 0 to 4142
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           4143 non-null   int64 
 1   state        4143 non-null   object
 2   coordinates  4143 non-null   object
 3   fuel type    4143 non-null   object
dtypes: int64(1), object(3)
memory usage: 129.6+ KB


In [104]:
# converting column values to string
# source: https://docs.python.org/3/library/ast.html#ast.literal_eval
top3_alt_fuels_NY_df['coordinates'] = top3_alt_fuels_NY_df['coordinates'].map(lambda x: ast.literal_eval(x))

In [105]:
# Rename the id, state, coordinates and fuel type columns.
top3_alt_fuels_NY_df = top3_alt_fuels_NY_df.rename(columns={"id":"ID", "state":"State", 
                                                            "coordinates":"Lon & Lat", "fuel type":"Fuel Type"})
top3_alt_fuels_NY_df.head()

,ID,State,Lon & Lat,Fuel Type
0,13401,NY,"[-74.672938, 41.651953]",LPG
1,13444,NY,"[-78.85922572883605, 42.8601082048398]",LPG
2,19234,NY,"[-73.634055, 40.658112]",LPG
3,19236,NY,"[-73.160001, 40.851029]",LPG
4,33001,NY,"[-73.464669, 40.692812]",E85


In [106]:
# Removing '[' from the beginning of 'Lat' values and ']' from the end of 'lon' values
for i, v in top3_alt_fuels_NY_df.iterrows():
    print(v['Lon & Lat'][0], v['Lon & Lat'][1])
    if i == 5:
        break;

-74.672938 41.651953
-78.85922572883605 42.8601082048398
-73.634055 40.658112
-73.160001 40.851029
-73.464669 40.692812
-73.99777 42.78657


In [107]:
# Separating the "Lat & Lon" column into the two columns of "latitude" and "longitude"
top3_alt_fuels_NY_df['Longitude'] = top3_alt_fuels_NY_df['Lon & Lat'].map(lambda x: x[0])
top3_alt_fuels_NY_df['Latitude'] = top3_alt_fuels_NY_df['Lon & Lat'].map(lambda x: x[1])
top3_alt_fuels_NY_df

,ID,State,Lon & Lat,Fuel Type,Longitude,Latitude
0,13401,NY,"[-74.672938, 41.651953]",LPG,-74.672938,41.651953
1,13444,NY,"[-78.85922572883605, 42.8601082048398]",LPG,-78.859226,42.860108
2,19234,NY,"[-73.634055, 40.658112]",LPG,-73.634055,40.658112
3,19236,NY,"[-73.160001, 40.851029]",LPG,-73.160001,40.851029
4,33001,NY,"[-73.464669, 40.692812]",E85,-73.464669,40.692812
...,...,...,...,...,...,...
4138,306301,NY,"[-73.15867, 40.86042]",ELEC,-73.158670,40.860420
4139,306302,NY,"[-73.15845, 40.860767]",ELEC,-73.158450,40.860767
4140,306335,NY,"[-72.755549, 40.956315]",ELEC,-72.755549,40.956315
4141,306336,NY,"[-72.755536, 40.956268]",ELEC,-72.755536,40.956268


In [108]:
# Changing order of "Longitude" and "Latitude" columns to "Latitude" and "Longitude"
top3_alt_fuels_NY_df = top3_alt_fuels_NY_df.reindex(['ID','State','Fuel Type','Latitude', 'Longitude'], axis=1)
top3_alt_fuels_NY_df

,ID,State,Fuel Type,Latitude,Longitude
0,13401,NY,LPG,41.651953,-74.672938
1,13444,NY,LPG,42.860108,-78.859226
2,19234,NY,LPG,40.658112,-73.634055
3,19236,NY,LPG,40.851029,-73.160001
4,33001,NY,E85,40.692812,-73.464669
...,...,...,...,...,...
4138,306301,NY,ELEC,40.860420,-73.158670
4139,306302,NY,ELEC,40.860767,-73.158450
4140,306335,NY,ELEC,40.956315,-72.755549
4141,306336,NY,ELEC,40.956268,-72.755536


In [109]:
# renaming "ELEC" to "Electric", "E85" to "Ethanol" and "LPG" to "Propane" in the "Fuel Type" column
top3_alt_fuels_NY_df['Fuel Type'] = top3_alt_fuels_NY_df['Fuel Type'].replace({'ELEC': 'Electric'})
top3_alt_fuels_NY_df['Fuel Type'] = top3_alt_fuels_NY_df['Fuel Type'].replace({'E85': 'Ethanol'})
top3_alt_fuels_NY_df['Fuel Type'] = top3_alt_fuels_NY_df['Fuel Type'].replace({'LPG': 'Propane'})
top3_alt_fuels_NY_df

,ID,State,Fuel Type,Latitude,Longitude
0,13401,NY,Propane,41.651953,-74.672938
1,13444,NY,Propane,42.860108,-78.859226
2,19234,NY,Propane,40.658112,-73.634055
3,19236,NY,Propane,40.851029,-73.160001
4,33001,NY,Ethanol,40.692812,-73.464669
...,...,...,...,...,...
4138,306301,NY,Electric,40.860420,-73.158670
4139,306302,NY,Electric,40.860767,-73.158450
4140,306335,NY,Electric,40.956315,-72.755549
4141,306336,NY,Electric,40.956268,-72.755536


In [110]:
# renaming "CA" to "California", "TX" to "Texas" and "NY" to "New York" in the "State" column
top3_alt_fuels_NY_df['State'] = top3_alt_fuels_NY_df['State'].replace({'CA': 'California'})
top3_alt_fuels_NY_df['State'] = top3_alt_fuels_NY_df['State'].replace({'TX': 'Texas'})
top3_alt_fuels_NY_df['State'] = top3_alt_fuels_NY_df['State'].replace({'NY': 'New York'})
top3_alt_fuels_NY_df

,ID,State,Fuel Type,Latitude,Longitude
0,13401,New York,Propane,41.651953,-74.672938
1,13444,New York,Propane,42.860108,-78.859226
2,19234,New York,Propane,40.658112,-73.634055
3,19236,New York,Propane,40.851029,-73.160001
4,33001,New York,Ethanol,40.692812,-73.464669
...,...,...,...,...,...
4138,306301,New York,Electric,40.860420,-73.158670
4139,306302,New York,Electric,40.860767,-73.158450
4140,306335,New York,Electric,40.956315,-72.755549
4141,306336,New York,Electric,40.956268,-72.755536


### Exporting Transformed Dataframe to CSV

In [111]:
# Export the newly transformed dataframe into a csv
top3_alt_fuels_NY_df.to_csv("../output_data/ETL/csv/ETL_top3_alt_fuels_NY.csv", index_label="#")

# Performing ETL on "E85_NY.csv" Dataset

## New York Ethanol Fuel Stations Data

In [112]:
# Create reference to CSV file
csv_path = Path("../output_data/data_analysis/csv/E85_NY.csv")

# Import the CSV into a pandas DataFrame
E85_NY_df = pd.read_csv(csv_path)
E85_NY_df

,id,state,coordinates,fuel type
0,33001,NY,"[-73.464669, 40.692812]",E85
1,33025,NY,"[-73.99777, 42.78657]",E85
2,33404,NY,"[-73.393473, 40.715264]",E85
3,33847,NY,"[-73.776371, 42.629724]",E85
4,33849,NY,"[-73.701367, 41.23846]",E85
...,...,...,...,...
69,224767,NY,"[-73.919457, 41.592159]",E85
70,226850,NY,"[-71.947876, 41.032781]",E85
71,236114,NY,"[-74.18958062957763, 41.15259135511587]",E85
72,238009,NY,"[-73.4913, 40.67424]",E85


In [113]:
# Get a brief summary of the E85_NY_df DataFrame.
E85_NY_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           74 non-null     int64 
 1   state        74 non-null     object
 2   coordinates  74 non-null     object
 3   fuel type    74 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.4+ KB


In [114]:
# converting column values to string
# source: https://docs.python.org/3/library/ast.html#ast.literal_eval
E85_NY_df['coordinates'] = E85_NY_df['coordinates'].map(lambda x: ast.literal_eval(x))

In [115]:
# Rename the id, state, coordinates and fuel type columns.
E85_NY_df = E85_NY_df.rename(columns={"id":"ID", "state":"State", 
                                      "coordinates":"Lon & Lat", "fuel type":"Fuel Type"})
E85_NY_df.head()

,ID,State,Lon & Lat,Fuel Type
0,33001,NY,"[-73.464669, 40.692812]",E85
1,33025,NY,"[-73.99777, 42.78657]",E85
2,33404,NY,"[-73.393473, 40.715264]",E85
3,33847,NY,"[-73.776371, 42.629724]",E85
4,33849,NY,"[-73.701367, 41.23846]",E85


In [116]:
# Removing '[' from the beginning of 'Lat' values and ']' from the end of 'lon' values
for i, v in E85_NY_df.iterrows():
    print(v['Lon & Lat'][0], v['Lon & Lat'][1])
    if i == 5:
        break;

-73.464669 40.692812
-73.99777 42.78657
-73.393473 40.715264
-73.776371 42.629724
-73.701367 41.23846
-76.094177 43.090389


In [117]:
# Separating the "Lat & Lon" column into the two columns of "latitude" and "longitude"
E85_NY_df['Longitude'] = E85_NY_df['Lon & Lat'].map(lambda x: x[0])
E85_NY_df['Latitude'] = E85_NY_df['Lon & Lat'].map(lambda x: x[1])
E85_NY_df

,ID,State,Lon & Lat,Fuel Type,Longitude,Latitude
0,33001,NY,"[-73.464669, 40.692812]",E85,-73.464669,40.692812
1,33025,NY,"[-73.99777, 42.78657]",E85,-73.997770,42.786570
2,33404,NY,"[-73.393473, 40.715264]",E85,-73.393473,40.715264
3,33847,NY,"[-73.776371, 42.629724]",E85,-73.776371,42.629724
4,33849,NY,"[-73.701367, 41.23846]",E85,-73.701367,41.238460
...,...,...,...,...,...,...
69,224767,NY,"[-73.919457, 41.592159]",E85,-73.919457,41.592159
70,226850,NY,"[-71.947876, 41.032781]",E85,-71.947876,41.032781
71,236114,NY,"[-74.18958062957763, 41.15259135511587]",E85,-74.189581,41.152591
72,238009,NY,"[-73.4913, 40.67424]",E85,-73.491300,40.674240


In [118]:
# Deleting the "Lon & Lat" column
E85_NY_df = E85_NY_df.drop('Lon & Lat', axis=1)
E85_NY_df

,ID,State,Fuel Type,Longitude,Latitude
0,33001,NY,E85,-73.464669,40.692812
1,33025,NY,E85,-73.997770,42.786570
2,33404,NY,E85,-73.393473,40.715264
3,33847,NY,E85,-73.776371,42.629724
4,33849,NY,E85,-73.701367,41.238460
...,...,...,...,...,...
69,224767,NY,E85,-73.919457,41.592159
70,226850,NY,E85,-71.947876,41.032781
71,236114,NY,E85,-74.189581,41.152591
72,238009,NY,E85,-73.491300,40.674240


In [119]:
# Changing order of "Longitude" and "Latitude" columns to "Latitude" and "Longitude"
E85_NY_df = E85_NY_df.reindex(['ID','State','Fuel Type','Latitude', 'Longitude'], axis=1)
E85_NY_df

,ID,State,Fuel Type,Latitude,Longitude
0,33001,NY,E85,40.692812,-73.464669
1,33025,NY,E85,42.786570,-73.997770
2,33404,NY,E85,40.715264,-73.393473
3,33847,NY,E85,42.629724,-73.776371
4,33849,NY,E85,41.238460,-73.701367
...,...,...,...,...,...
69,224767,NY,E85,41.592159,-73.919457
70,226850,NY,E85,41.032781,-71.947876
71,236114,NY,E85,41.152591,-74.189581
72,238009,NY,E85,40.674240,-73.491300


In [120]:
# renaming "ELEC" to "Electric", "E85" to "Ethanol" and "LPG" to "Propane" in the "Fuel Type" column
E85_NY_df['Fuel Type'] = E85_NY_df['Fuel Type'].replace({'ELEC': 'Electric'})
E85_NY_df['Fuel Type'] = E85_NY_df['Fuel Type'].replace({'E85': 'Ethanol'})
E85_NY_df['Fuel Type'] = E85_NY_df['Fuel Type'].replace({'LPG': 'Propane'})
E85_NY_df

,ID,State,Fuel Type,Latitude,Longitude
0,33001,NY,Ethanol,40.692812,-73.464669
1,33025,NY,Ethanol,42.786570,-73.997770
2,33404,NY,Ethanol,40.715264,-73.393473
3,33847,NY,Ethanol,42.629724,-73.776371
4,33849,NY,Ethanol,41.238460,-73.701367
...,...,...,...,...,...
69,224767,NY,Ethanol,41.592159,-73.919457
70,226850,NY,Ethanol,41.032781,-71.947876
71,236114,NY,Ethanol,41.152591,-74.189581
72,238009,NY,Ethanol,40.674240,-73.491300


In [121]:
# renaming "CA" to "California", "TX" to "Texas" and "NY" to "New York" in the "State" column
E85_NY_df['State'] = E85_NY_df['State'].replace({'CA': 'California'})
E85_NY_df['State'] = E85_NY_df['State'].replace({'TX': 'Texas'})
E85_NY_df['State'] = E85_NY_df['State'].replace({'NY': 'New York'})
E85_NY_df

,ID,State,Fuel Type,Latitude,Longitude
0,33001,New York,Ethanol,40.692812,-73.464669
1,33025,New York,Ethanol,42.786570,-73.997770
2,33404,New York,Ethanol,40.715264,-73.393473
3,33847,New York,Ethanol,42.629724,-73.776371
4,33849,New York,Ethanol,41.238460,-73.701367
...,...,...,...,...,...
69,224767,New York,Ethanol,41.592159,-73.919457
70,226850,New York,Ethanol,41.032781,-71.947876
71,236114,New York,Ethanol,41.152591,-74.189581
72,238009,New York,Ethanol,40.674240,-73.491300


### Exporting Transformed Dataframe to CSV

In [122]:
# Export the newly transformed dataframe into a csv
E85_NY_df.to_csv("../output_data/ETL/csv/ETL_E85_NY.csv", index_label="#")

# Performing ETL on "ELEC_NY.csv" Dataset

## New York Electric Car Recharging Stations Data

In [123]:
# Create reference to CSV file
csv_path = Path("../output_data/data_analysis/csv/ELEC_NY.csv")

# Import the CSV into a pandas DataFrame
ELEC_NY_df = pd.read_csv(csv_path)
ELEC_NY_df

,id,state,coordinates,fuel type
0,35272,NY,"[-73.608978, 40.814019]",ELEC
1,35562,NY,"[-73.857693, 42.702424]",ELEC
2,35563,NY,"[-73.773021, 42.969823]",ELEC
3,36532,NY,"[-73.774182, 42.97167]",ELEC
4,41240,NY,"[-73.906405, 40.867403]",ELEC
...,...,...,...,...
4015,306301,NY,"[-73.15867, 40.86042]",ELEC
4016,306302,NY,"[-73.15845, 40.860767]",ELEC
4017,306335,NY,"[-72.755549, 40.956315]",ELEC
4018,306336,NY,"[-72.755536, 40.956268]",ELEC


In [124]:
# Get a brief summary of the ELEC_NY_df DataFrame.
ELEC_NY_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4020 entries, 0 to 4019
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           4020 non-null   int64 
 1   state        4020 non-null   object
 2   coordinates  4020 non-null   object
 3   fuel type    4020 non-null   object
dtypes: int64(1), object(3)
memory usage: 125.8+ KB


In [125]:
# converting column values to string
# source: https://docs.python.org/3/library/ast.html#ast.literal_eval
ELEC_NY_df['coordinates'] = ELEC_NY_df['coordinates'].map(lambda x: ast.literal_eval(x))

In [126]:
# Rename the id, state, coordinates and fuel type columns.
ELEC_NY_df = ELEC_NY_df.rename(columns={"id":"ID", "state":"State", 
                                      "coordinates":"Lon & Lat", "fuel type":"Fuel Type"})
ELEC_NY_df.head()

,ID,State,Lon & Lat,Fuel Type
0,35272,NY,"[-73.608978, 40.814019]",ELEC
1,35562,NY,"[-73.857693, 42.702424]",ELEC
2,35563,NY,"[-73.773021, 42.969823]",ELEC
3,36532,NY,"[-73.774182, 42.97167]",ELEC
4,41240,NY,"[-73.906405, 40.867403]",ELEC


In [127]:
# Removing '[' from the beginning of 'Lat' values and ']' from the end of 'lon' values
for i, v in ELEC_NY_df.iterrows():
    print(v['Lon & Lat'][0], v['Lon & Lat'][1])
    if i == 5:
        break;

-73.608978 40.814019
-73.857693 42.702424
-73.773021 42.969823
-73.774182 42.97167
-73.906405 40.867403
-74.068916 41.504105


In [128]:
# Separating the "Lat & Lon" column into the two columns of "latitude" and "longitude"
ELEC_NY_df['Longitude'] = ELEC_NY_df['Lon & Lat'].map(lambda x: x[0])
ELEC_NY_df['Latitude'] = ELEC_NY_df['Lon & Lat'].map(lambda x: x[1])
ELEC_NY_df

,ID,State,Lon & Lat,Fuel Type,Longitude,Latitude
0,35272,NY,"[-73.608978, 40.814019]",ELEC,-73.608978,40.814019
1,35562,NY,"[-73.857693, 42.702424]",ELEC,-73.857693,42.702424
2,35563,NY,"[-73.773021, 42.969823]",ELEC,-73.773021,42.969823
3,36532,NY,"[-73.774182, 42.97167]",ELEC,-73.774182,42.971670
4,41240,NY,"[-73.906405, 40.867403]",ELEC,-73.906405,40.867403
...,...,...,...,...,...,...
4015,306301,NY,"[-73.15867, 40.86042]",ELEC,-73.158670,40.860420
4016,306302,NY,"[-73.15845, 40.860767]",ELEC,-73.158450,40.860767
4017,306335,NY,"[-72.755549, 40.956315]",ELEC,-72.755549,40.956315
4018,306336,NY,"[-72.755536, 40.956268]",ELEC,-72.755536,40.956268


In [129]:
# Deleting the "Lon & Lat" column
ELEC_NY_df = ELEC_NY_df.drop('Lon & Lat', axis=1)
ELEC_NY_df

,ID,State,Fuel Type,Longitude,Latitude
0,35272,NY,ELEC,-73.608978,40.814019
1,35562,NY,ELEC,-73.857693,42.702424
2,35563,NY,ELEC,-73.773021,42.969823
3,36532,NY,ELEC,-73.774182,42.971670
4,41240,NY,ELEC,-73.906405,40.867403
...,...,...,...,...,...
4015,306301,NY,ELEC,-73.158670,40.860420
4016,306302,NY,ELEC,-73.158450,40.860767
4017,306335,NY,ELEC,-72.755549,40.956315
4018,306336,NY,ELEC,-72.755536,40.956268


In [130]:
# Changing order of "Longitude" and "Latitude" columns to "Latitude" and "Longitude"
ELEC_NY_df = ELEC_NY_df.reindex(['ID','State','Fuel Type','Latitude', 'Longitude'], axis=1)
ELEC_NY_df

,ID,State,Fuel Type,Latitude,Longitude
0,35272,NY,ELEC,40.814019,-73.608978
1,35562,NY,ELEC,42.702424,-73.857693
2,35563,NY,ELEC,42.969823,-73.773021
3,36532,NY,ELEC,42.971670,-73.774182
4,41240,NY,ELEC,40.867403,-73.906405
...,...,...,...,...,...
4015,306301,NY,ELEC,40.860420,-73.158670
4016,306302,NY,ELEC,40.860767,-73.158450
4017,306335,NY,ELEC,40.956315,-72.755549
4018,306336,NY,ELEC,40.956268,-72.755536


In [131]:
# renaming "ELEC" to "Electric", "E85" to "Ethanol" and "LPG" to "Propane" in the "Fuel Type" column
ELEC_NY_df['Fuel Type'] = ELEC_NY_df['Fuel Type'].replace({'ELEC': 'Electric'})
ELEC_NY_df['Fuel Type'] = ELEC_NY_df['Fuel Type'].replace({'E85': 'Ethanol'})
ELEC_NY_df['Fuel Type'] = ELEC_NY_df['Fuel Type'].replace({'LPG': 'Propane'})
ELEC_NY_df

,ID,State,Fuel Type,Latitude,Longitude
0,35272,NY,Electric,40.814019,-73.608978
1,35562,NY,Electric,42.702424,-73.857693
2,35563,NY,Electric,42.969823,-73.773021
3,36532,NY,Electric,42.971670,-73.774182
4,41240,NY,Electric,40.867403,-73.906405
...,...,...,...,...,...
4015,306301,NY,Electric,40.860420,-73.158670
4016,306302,NY,Electric,40.860767,-73.158450
4017,306335,NY,Electric,40.956315,-72.755549
4018,306336,NY,Electric,40.956268,-72.755536


In [132]:
# renaming "CA" to "California", "TX" to "Texas" and "NY" to "New York" in the "State" column
ELEC_NY_df['State'] = ELEC_NY_df['State'].replace({'CA': 'California'})
ELEC_NY_df['State'] = ELEC_NY_df['State'].replace({'TX': 'Texas'})
ELEC_NY_df['State'] = ELEC_NY_df['State'].replace({'NY': 'New York'})
ELEC_NY_df

,ID,State,Fuel Type,Latitude,Longitude
0,35272,New York,Electric,40.814019,-73.608978
1,35562,New York,Electric,42.702424,-73.857693
2,35563,New York,Electric,42.969823,-73.773021
3,36532,New York,Electric,42.971670,-73.774182
4,41240,New York,Electric,40.867403,-73.906405
...,...,...,...,...,...
4015,306301,New York,Electric,40.860420,-73.158670
4016,306302,New York,Electric,40.860767,-73.158450
4017,306335,New York,Electric,40.956315,-72.755549
4018,306336,New York,Electric,40.956268,-72.755536


### Exporting Transformed Dataframe to CSV

In [133]:
# Export the newly transformed dataframe into a csv
ELEC_NY_df.to_csv("../output_data/ETL/csv/ETL_ELEC_NY.csv", index_label="#")

# Performing ETL on "LPG_NY.csv" Dataset

## New York Liquid Propane Gas Stations Data

In [134]:
# Create reference to CSV file
csv_path = Path("../output_data/data_analysis/csv/LPG_NY.csv")

# Import the CSV into a pandas DataFrame
LPG_NY_df = pd.read_csv(csv_path)
LPG_NY_df

,id,state,coordinates,fuel type
0,13401,NY,"[-74.672938, 41.651953]",LPG
1,13444,NY,"[-78.85922572883605, 42.8601082048398]",LPG
2,19234,NY,"[-73.634055, 40.658112]",LPG
3,19236,NY,"[-73.160001, 40.851029]",LPG
4,33486,NY,"[-76.6744609, 42.5497889]",LPG
5,36400,NY,"[-73.558148, 40.780796]",LPG
6,41110,NY,"[-73.580334, 42.907221]",LPG
7,41111,NY,"[-73.855453, 42.982234]",LPG
8,42856,NY,"[-77.65186, 43.120818]",LPG
9,50098,NY,"[-74.384605, 41.397445]",LPG


In [135]:
# Get a brief summary of the LPG_CA_df DataFrame.
LPG_NY_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           49 non-null     int64 
 1   state        49 non-null     object
 2   coordinates  49 non-null     object
 3   fuel type    49 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.7+ KB


In [136]:
# converting column values to string
# source: https://docs.python.org/3/library/ast.html#ast.literal_eval
LPG_NY_df['coordinates'] = LPG_NY_df['coordinates'].map(lambda x: ast.literal_eval(x))

In [137]:
# Rename the id, state, coordinates and fuel type columns.
LPG_NY_df = LPG_NY_df.rename(columns={"id":"ID", "state":"State", 
                                      "coordinates":"Lon & Lat", "fuel type":"Fuel Type"})
LPG_NY_df.head()

,ID,State,Lon & Lat,Fuel Type
0,13401,NY,"[-74.672938, 41.651953]",LPG
1,13444,NY,"[-78.85922572883605, 42.8601082048398]",LPG
2,19234,NY,"[-73.634055, 40.658112]",LPG
3,19236,NY,"[-73.160001, 40.851029]",LPG
4,33486,NY,"[-76.6744609, 42.5497889]",LPG


In [138]:
# Removing '[' from the beginning of 'Lat' values and ']' from the end of 'lon' values
for i, v in LPG_NY_df.iterrows():
    print(v['Lon & Lat'][0], v['Lon & Lat'][1])
    if i == 5:
        break;

-74.672938 41.651953
-78.85922572883605 42.8601082048398
-73.634055 40.658112
-73.160001 40.851029
-76.6744609 42.5497889
-73.558148 40.780796


In [139]:
# Separating the "Lat & Lon" column into the two columns of "latitude" and "longitude"
LPG_NY_df['Longitude'] = LPG_NY_df['Lon & Lat'].map(lambda x: x[0])
LPG_NY_df['Latitude'] = LPG_NY_df['Lon & Lat'].map(lambda x: x[1])
LPG_NY_df

,ID,State,Lon & Lat,Fuel Type,Longitude,Latitude
0,13401,NY,"[-74.672938, 41.651953]",LPG,-74.672938,41.651953
1,13444,NY,"[-78.85922572883605, 42.8601082048398]",LPG,-78.859226,42.860108
2,19234,NY,"[-73.634055, 40.658112]",LPG,-73.634055,40.658112
3,19236,NY,"[-73.160001, 40.851029]",LPG,-73.160001,40.851029
4,33486,NY,"[-76.6744609, 42.5497889]",LPG,-76.674461,42.549789
5,36400,NY,"[-73.558148, 40.780796]",LPG,-73.558148,40.780796
6,41110,NY,"[-73.580334, 42.907221]",LPG,-73.580334,42.907221
7,41111,NY,"[-73.855453, 42.982234]",LPG,-73.855453,42.982234
8,42856,NY,"[-77.65186, 43.120818]",LPG,-77.651860,43.120818
9,50098,NY,"[-74.384605, 41.397445]",LPG,-74.384605,41.397445


In [140]:
# Deleting the "Lon & Lat" column
LPG_NY_df = LPG_NY_df.drop('Lon & Lat', axis=1)
LPG_NY_df

,ID,State,Fuel Type,Longitude,Latitude
0,13401,NY,LPG,-74.672938,41.651953
1,13444,NY,LPG,-78.859226,42.860108
2,19234,NY,LPG,-73.634055,40.658112
3,19236,NY,LPG,-73.160001,40.851029
4,33486,NY,LPG,-76.674461,42.549789
5,36400,NY,LPG,-73.558148,40.780796
6,41110,NY,LPG,-73.580334,42.907221
7,41111,NY,LPG,-73.855453,42.982234
8,42856,NY,LPG,-77.651860,43.120818
9,50098,NY,LPG,-74.384605,41.397445


In [141]:
# Changing order of "Longitude" and "Latitude" columns to "Latitude" and "Longitude"
LPG_NY_df = LPG_NY_df.reindex(['ID','State','Fuel Type','Latitude', 'Longitude'], axis=1)
LPG_NY_df

,ID,State,Fuel Type,Latitude,Longitude
0,13401,NY,LPG,41.651953,-74.672938
1,13444,NY,LPG,42.860108,-78.859226
2,19234,NY,LPG,40.658112,-73.634055
3,19236,NY,LPG,40.851029,-73.160001
4,33486,NY,LPG,42.549789,-76.674461
5,36400,NY,LPG,40.780796,-73.558148
6,41110,NY,LPG,42.907221,-73.580334
7,41111,NY,LPG,42.982234,-73.855453
8,42856,NY,LPG,43.120818,-77.651860
9,50098,NY,LPG,41.397445,-74.384605


In [142]:
# renaming "ELEC" to "Electric", "E85" to "Ethanol" and "LPG" to "Propane" in the "Fuel Type" column
LPG_NY_df['Fuel Type'] = LPG_NY_df['Fuel Type'].replace({'ELEC': 'Electric'})
LPG_NY_df['Fuel Type'] = LPG_NY_df['Fuel Type'].replace({'E85': 'Ethanol'})
LPG_NY_df['Fuel Type'] = LPG_NY_df['Fuel Type'].replace({'LPG': 'Propane'})
LPG_NY_df

,ID,State,Fuel Type,Latitude,Longitude
0,13401,NY,Propane,41.651953,-74.672938
1,13444,NY,Propane,42.860108,-78.859226
2,19234,NY,Propane,40.658112,-73.634055
3,19236,NY,Propane,40.851029,-73.160001
4,33486,NY,Propane,42.549789,-76.674461
5,36400,NY,Propane,40.780796,-73.558148
6,41110,NY,Propane,42.907221,-73.580334
7,41111,NY,Propane,42.982234,-73.855453
8,42856,NY,Propane,43.120818,-77.651860
9,50098,NY,Propane,41.397445,-74.384605


In [143]:
# renaming "CA" to "California", "TX" to "Texas" and "NY" to "New York" in the "State" column
LPG_NY_df['State'] = LPG_NY_df['State'].replace({'CA': 'California'})
LPG_NY_df['State'] = LPG_NY_df['State'].replace({'TX': 'Texas'})
LPG_NY_df['State'] = LPG_NY_df['State'].replace({'NY': 'New York'})
LPG_NY_df

,ID,State,Fuel Type,Latitude,Longitude
0,13401,New York,Propane,41.651953,-74.672938
1,13444,New York,Propane,42.860108,-78.859226
2,19234,New York,Propane,40.658112,-73.634055
3,19236,New York,Propane,40.851029,-73.160001
4,33486,New York,Propane,42.549789,-76.674461
5,36400,New York,Propane,40.780796,-73.558148
6,41110,New York,Propane,42.907221,-73.580334
7,41111,New York,Propane,42.982234,-73.855453
8,42856,New York,Propane,43.120818,-77.651860
9,50098,New York,Propane,41.397445,-74.384605


### Exporting Transformed Dataframe to CSV

In [144]:
# Export the newly transformed dataframe into a csv
LPG_NY_df.to_csv("../output_data/ETL/csv/ETL_LPG_NY.csv", index_label="#")